In [89]:
import requests

# Your credentials
consumer_key = " 0xdYfz2zFpkbpsgNLnM75LIEt8ayfjzyTzeL00g45m9814np"
consumer_secret = "TV8jBWPaGwV3iqHkZuKnVmMKlLOYxpQgG9DcdE7Zbij5ROhLWsUfNiqUzklQAylG"

# Get access token
token_url = "https://ops.epo.org/3.2/auth/accesstoken"
token_response = requests.post(
    token_url,
    data={
        "grant_type": "client_credentials",
        "client_id": consumer_key,
        "client_secret": consumer_secret
    },
    headers={"Content-Type": "application/x-www-form-urlencoded"}
)

access_token = token_response.json()["access_token"]

# Make API request
api_url = "https://ops.epo.org/3.2/rest-services/published-data/search?Range=1-25&q=ti%3Dplastic"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json"
}

response = requests.get(api_url, headers=headers)
print(response.json())

{'ops:world-patent-data': {'@xmlns': {'ops': 'http://ops.epo.org', '$': 'http://www.epo.org/exchange', 'xlink': 'http://www.w3.org/1999/xlink'}, 'ops:biblio-search': {'@total-result-count': '10000', 'ops:query': {'$': 'ti = plastic', '@syntax': 'CQL'}, 'ops:range': {'@begin': '1', '@end': '25'}, 'ops:search-result': {'ops:publication-reference': [{'@system': 'ops.epo.org', '@family-id': '95211732', 'document-id': {'@document-id-type': 'docdb', 'country': {'$': 'PH'}, 'doc-number': {'$': '22023050866'}, 'kind': {'$': 'U1'}}}, {'@system': 'ops.epo.org', '@family-id': '95206969', 'document-id': {'@document-id-type': 'docdb', 'country': {'$': 'KR'}, 'doc-number': {'$': '20250041856'}, 'kind': {'$': 'A'}}}, {'@system': 'ops.epo.org', '@family-id': '95206338', 'document-id': {'@document-id-type': 'docdb', 'country': {'$': 'KR'}, 'doc-number': {'$': '102781814'}, 'kind': {'$': 'B1'}}}, {'@system': 'ops.epo.org', '@family-id': '95206145', 'document-id': {'@document-id-type': 'docdb', 'country'

In [4]:
import requests

# Your credentials
consumer_key = " 0xdYfz2zFpkbpsgNLnM75LIEt8ayfjzyTzeL00g45m9814np"
consumer_secret = "TV8jBWPaGwV3iqHkZuKnVmMKlLOYxpQgG9DcdE7Zbij5ROhLWsUfNiqUzklQAylG"

# Get access token
token_url = "https://ops.epo.org/3.2/auth/accesstoken"
token_response = requests.post(
    token_url,
    data={
        "grant_type": "client_credentials",
        "client_id": consumer_key,
        "client_secret": consumer_secret
    },
    headers={"Content-Type": "application/x-www-form-urlencoded"}
)

access_token = token_response.json()["access_token"]

# Make API request
api_url = "https://ops.epo.org/3.2/rest-services/family/publication/docdb/US2014006294A1%20"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json"
}

response = requests.get(api_url, headers=headers)
response.json()

{'ops:world-patent-data': {'@xmlns': {'ops': 'http://ops.epo.org',
   '$': 'http://www.epo.org/exchange',
   'xlink': 'http://www.w3.org/1999/xlink'},
  'ops:patent-family': {'@legal': 'false',
   '@total-result-count': '5',
   'ops:publication-reference': {'document-id': {'@document-id-type': 'docdb',
     'country': {'$': 'US'},
     'doc-number': {'$': '2014006294'},
     'kind': {'$': 'A1'}}},
   'ops:family-member': [{'@family-id': '49779178',
     'publication-reference': {'document-id': [{'@document-id-type': 'docdb',
        'country': {'$': 'US'},
        'doc-number': {'$': '2014006294'},
        'kind': {'$': 'A1'},
        'date': {'$': '20140102'}},
       {'@document-id-type': 'epodoc',
        'doc-number': {'$': 'US2014006294'},
        'date': {'$': '20140102'}}]},
     'application-reference': {'@doc-id': '413591169',
      '@is-representative': 'YES',
      'document-id': {'@document-id-type': 'docdb',
       'country': {'$': 'US'},
       'doc-number': {'$': '201313

search query , retrieving publication numbers

In [94]:
import requests
import xml.etree.ElementTree as ET
import json
import time
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

def get_access_token():
    """Get OAuth access token using credentials from .env"""
    token_url = "https://ops.epo.org/3.2/auth/accesstoken"
    response = requests.post(
        token_url,
        data={
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    response.raise_for_status()
    return response.json()["access_token"]

# Obtain the access token
access_token = get_access_token()
print("Access token obtained.")

# Set up the API endpoint, headers, and static query
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/xml"
}
base_url = "https://ops.epo.org/3.2/rest-services/published-data/search"
query = "ti=plastic"  # static query

doc_numbers = []

# Define XML namespaces for proper parsing.
namespaces = {
    'ops': 'http://ops.epo.org',         # for ops-prefixed elements
    'ex': 'http://www.epo.org/exchange'   # for elements in the default namespace (e.g., <doc-number>)
}

# Loop over ranges to collect up to 500 document numbers (25 per call)
for start in range(1, 500, 25):
    end = start + 24
    params = {
        "Range": f"{start}-{end}",
        "q": query
    }
    print(f"Requesting results for range {start}-{end}...")
    
    response = requests.get(base_url, params=params, headers=headers)
    
    if response.status_code != 200:
        print(f"Error {response.status_code} for range {start}-{end}")
        continue
    
    try:
        root = ET.fromstring(response.content)
    except ET.ParseError as err:
        print(f"XML parse error for range {start}-{end}: {err}")
        continue

    # Extract document numbers from each publication-reference element
    for pub_ref in root.findall(".//ops:publication-reference", namespaces):
        doc_num_elem = pub_ref.find(".//ex:doc-number", namespaces)
        if doc_num_elem is not None and doc_num_elem.text:
            doc_numbers.append(doc_num_elem.text.strip())
    
    time.sleep(1)  # Pause between requests

# Remove duplicates and sort the list of document numbers
doc_numbers = sorted(list(set(doc_numbers)))

# Output the results as JSON (saving to file)
output_filename = "epo_doc_numbers.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(doc_numbers, f, indent=2)

print(f"Retrieved {len(doc_numbers)} unique document numbers.")
print(json.dumps(doc_numbers, indent=2))


Access token obtained.
Requesting results for range 1-25...
Requesting results for range 26-50...
Requesting results for range 51-75...
Requesting results for range 76-100...
Requesting results for range 101-125...
Requesting results for range 126-150...
Requesting results for range 151-175...
Requesting results for range 176-200...
Requesting results for range 201-225...
Requesting results for range 226-250...
Requesting results for range 251-275...
Requesting results for range 276-300...
Requesting results for range 301-325...
Requesting results for range 326-350...
Requesting results for range 351-375...
Requesting results for range 376-400...
Requesting results for range 401-425...
Requesting results for range 426-450...
Requesting results for range 451-475...
Requesting results for range 476-500...
Retrieved 500 unique document numbers.
[
  "102760632",
  "102764616",
  "102764837",
  "102766312",
  "102766628",
  "102766743",
  "102767595",
  "102770410",
  "102770567",
  "102776

In [97]:
import requests
import xml.etree.ElementTree as ET
import json
import time
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

def get_access_token():
    """Get OAuth access token using credentials from .env"""
    token_url = "https://ops.epo.org/3.2/auth/accesstoken"
    response = requests.post(
        token_url,
        data={
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    response.raise_for_status()
    return response.json()["access_token"]

# Obtain the access token
access_token = get_access_token()
print("Access token obtained.")

# Set up the API endpoint, headers, and static query
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/xml"
}
base_url = "https://ops.epo.org/3.2/rest-services/published-data/search"
query = "ti=plastic"  # static query

publication_numbers = []  # List to store publication numbers

# Define XML namespaces for proper parsing.
namespaces = {
    'ops': 'http://ops.epo.org',         # For ops-prefixed elements
    'ex': 'http://www.epo.org/exchange'   # For elements in the default namespace (e.g., <country>, <doc-number>, <kind>)
}

# Loop over ranges to collect up to 500 results (25 per call)
for start in range(1, 500, 50):
    end = start + 24
    params = {
        "Range": f"{start}-{end}",
        "q": query
    }
    print(f"Requesting results for range {start}-{end}...")
    
    response = requests.get(base_url, params=params, headers=headers)
    
    if response.status_code != 200:
        print(f"Error {response.status_code} for range {start}-{end}")
        continue
    
    try:
        root = ET.fromstring(response.content)
    except ET.ParseError as err:
        print(f"XML parse error for range {start}-{end}: {err}")
        continue

    # For each publication-reference element, extract country, doc-number, and kind
    for pub_ref in root.findall(".//ops:publication-reference", namespaces):
        doc_id = pub_ref.find("ex:document-id", namespaces)
        if doc_id is None:
            continue
        country_elem = doc_id.find("ex:country", namespaces)
        doc_number_elem = doc_id.find("ex:doc-number", namespaces)
        kind_elem = doc_id.find("ex:kind", namespaces)
        
        if country_elem is not None and doc_number_elem is not None and kind_elem is not None:
            country = country_elem.text.strip() if country_elem.text else ""
            doc_number = doc_number_elem.text.strip() if doc_number_elem.text else ""
            kind = kind_elem.text.strip() if kind_elem.text else ""
            # Combine values into one string with no spaces: publication number
            publication_number = f"{country}{doc_number}{kind}"
            publication_numbers.append(publication_number)
    
    time.sleep(1)  # Pause between requests

# Remove duplicates and sort the publication numbers
publication_numbers = sorted(list(set(publication_numbers)))

# Save the publication numbers as JSON
output_filename = "epo_publication_numbers.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(publication_numbers, f, indent=2)

print(f"Retrieved {len(publication_numbers)} unique publication numbers.")
print(json.dumps(publication_numbers, indent=2))


Access token obtained.
Requesting results for range 1-25...
Requesting results for range 51-75...
Requesting results for range 101-125...
Requesting results for range 151-175...
Requesting results for range 201-225...
Requesting results for range 251-275...
Requesting results for range 301-325...
Requesting results for range 351-375...
Requesting results for range 401-425...
Requesting results for range 451-475...
Retrieved 250 unique publication numbers.
[
  "AU2023329079A1",
  "CR20240476A",
  "DK4269498T3",
  "ES1314112U",
  "ES2999767T3",
  "ES3000087T3",
  "ES3001908T3",
  "ES3004573T3",
  "ES3007215T3",
  "ES3010164T3",
  "HUE069562T2",
  "KR102760632B1",
  "KR102767595B1",
  "KR102770410B1",
  "KR102776548B1",
  "KR102777727B1",
  "KR102777728B1",
  "KR102777733B1",
  "KR102777734B1",
  "KR102781813B1",
  "KR102781814B1",
  "KR102783082B1",
  "KR20250014124A",
  "KR20250014874A",
  "KR20250022932A",
  "KR20250025074A",
  "KR20250028575A",
  "KR20250028670A",
  "KR20250028771A",


In [ ]:
import requests
import xml.etree.ElementTree as ET
import json
import time
import os
from dotenv import load_dotenv
import pandas as pd
# Load environment variables from .env file
load_dotenv()

CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

def get_access_token():
    """Get OAuth access token using credentials from .env"""
    token_url = "https://ops.epo.org/3.2/auth/accesstoken"
    response = requests.post(
        token_url,
        data={
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    response.raise_for_status()
    return response.json()["access_token"]

# Obtain the access token
access_token = get_access_token()
print("Access token obtained.")

# Set up the API endpoint, headers, and static query
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/xml"
}
base_url = "https://ops.epo.org/3.2/rest-services/published-data/search"
query = "ti=plastic"  # static query


# We'll store each record as a dictionary with keys: "publication_num" and "country"
records = []

# Define XML namespaces for proper parsing.
namespaces = {
    'ops': 'http://ops.epo.org',         # For ops-prefixed elements
    'ex': 'http://www.epo.org/exchange'   # For elements in the default namespace (e.g., <country>, <doc-number>, <kind>)
}

# Loop over ranges to cover up to 500 results (25 per batch)
for start in range(1, 500, 56):
    end = start + 55
    params = {
        "Range": f"{start}-{end}",
        "q": query
    }
    print(f"Requesting results for range {start}-{end}...")
    
    response = requests.get(base_url, params=params, headers=headers)
    
    if response.status_code != 200:
        print(f"Error {response.status_code} for range {start}-{end}")
        continue
    
    try:
        root = ET.fromstring(response.content)
    except ET.ParseError as err:
        print(f"XML parse error for range {start}-{end}: {err}")
        continue

    # For each publication-reference element, extract country, doc-number, and kind
    for pub_ref in root.findall(".//ops:publication-reference", namespaces):
        doc_id = pub_ref.find("ex:document-id", namespaces)
        if doc_id is None:
            continue
        
        country_elem = doc_id.find("ex:country", namespaces)
        doc_number_elem = doc_id.find("ex:doc-number", namespaces)
        kind_elem = doc_id.find("ex:kind", namespaces)
        
        if country_elem is not None and doc_number_elem is not None and kind_elem is not None:
            country = country_elem.text.strip() if country_elem.text else ""
            doc_number = doc_number_elem.text.strip() if doc_number_elem.text else ""
            kind = kind_elem.text.strip() if kind_elem.text else ""
            # Concatenate values (no spaces) to form publication_num
            publication_num = f"{country}{doc_number}{kind}"
            records.append({
                "publication_num": publication_num,
                "country": country
            })
    
    time.sleep(1)  # Pause between requests

# Convert the list of records to a DataFrame
df = pd.DataFrame(records)

# Optional: Remove duplicates and sort the DataFrame
df.drop_duplicates(inplace=True)
df.sort_values(by="publication_num", inplace=True)

# Print the DataFrame
df.head()

# Optionally, export to Excel or CSV if needed:
# df.to_excel("epo_publications.xlsx", index=False)
# or
# df.to_csv("epo_publications.csv", index=False)


Access token obtained.
Requesting results for range 1-56...
Requesting results for range 57-112...
Requesting results for range 113-168...
Requesting results for range 169-224...
Requesting results for range 225-280...
Requesting results for range 281-336...
Requesting results for range 337-392...
Requesting results for range 393-448...
Requesting results for range 449-504...


,publication_num,country
165,AU2023329079A1,AU
87,AU2023345802A1,AU
85,AU2023369720A1,AU
86,AU2024219494A1,AU
83,AU2024219543A1,AU


In [131]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import os
from dotenv import load_dotenv
import time

# Load environment variables from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

def get_access_token():
    """Get OAuth access token using credentials from .env"""
    token_url = "https://ops.epo.org/3.2/auth/accesstoken"
    response = requests.post(
        token_url,
        data={
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    response.raise_for_status()
    return response.json()["access_token"]

def parse_xml_to_records(xml_content):
    """Parse XML response and return list of dictionaries with patent info"""
    records = []
    try:
        root = ET.fromstring(xml_content)
    except ET.ParseError as err:
        print("Error parsing XML:", err)
        return records

    for doc in root.findall(".//{http://www.epo.org/exchange}exchange-document"):
        record = {
            "publication_number": "",
            "country": "",
            "publication_date": "",
            "priority_date": "",
            "applicants": "",
            "inventors": "",
            "title": "",
            "abstract": "",
            "ipc_classification": ""
        }

        pub_ref = doc.find(".//{http://www.epo.org/exchange}publication-reference")
        if pub_ref is not None:
            docdb = pub_ref.find(".//{http://www.epo.org/exchange}document-id[@document-id-type='docdb']")
            if docdb is not None:
                country = docdb.find("{http://www.epo.org/exchange}country")
                docnum = docdb.find("{http://www.epo.org/exchange}doc-number")
                kind = docdb.find("{http://www.epo.org/exchange}kind")
                date = docdb.find("{http://www.epo.org/exchange}date")
                if country is not None and docnum is not None and kind is not None:
                    record["country"] = country.text.strip()
                    record["publication_number"] = f"{country.text.strip()}{docnum.text.strip()}{kind.text.strip()}"
                if date is not None:
                    record["publication_date"] = date.text.strip()

        prios = doc.find(".//{http://www.epo.org/exchange}priority-claims")
        if prios is not None:
            prio = prios.find("{http://www.epo.org/exchange}priority-claim")
            if prio is not None:
                prio_docid = prio.find("{http://www.epo.org/exchange}document-id")
                if prio_docid is not None:
                    prio_date = prio_docid.find("{http://www.epo.org/exchange}date")
                    if prio_date is not None:
                        record["priority_date"] = prio_date.text.strip()

        parties = doc.find(".//{http://www.epo.org/exchange}parties")
        if parties is not None:
            applicants = parties.find("{http://www.epo.org/exchange}applicants")
            inventors = parties.find("{http://www.epo.org/exchange}inventors")
            applicant_names = []
            inventor_names = []
            if applicants is not None:
                for applicant in applicants.findall("{http://www.epo.org/exchange}applicant"):
                    name = applicant.find(".//{http://www.epo.org/exchange}name")
                    if name is not None and name.text:
                        applicant_names.append(name.text.strip())
            if inventors is not None:
                for inventor in inventors.findall("{http://www.epo.org/exchange}inventor"):
                    name = inventor.find(".//{http://www.epo.org/exchange}name")
                    if name is not None and name.text:
                        inventor_names.append(name.text.strip())
            record["applicants"] = "; ".join(applicant_names)
            record["inventors"] = "; ".join(inventor_names)

        title_elem = doc.find(".//{http://www.epo.org/exchange}invention-title[@lang='en']")
        if title_elem is None:
            title_elem = doc.find(".//{http://www.epo.org/exchange}invention-title")
        if title_elem is not None and title_elem.text:
            record["title"] = title_elem.text.strip()

        abstract = doc.find(".//{http://www.epo.org/exchange}abstract")
        if abstract is not None:
            paras = abstract.findall("{http://www.epo.org/exchange}p")
            if paras:
                record["abstract"] = " ".join(p.text.strip() for p in paras if p.text)
            elif abstract.text:
                record["abstract"] = abstract.text.strip()

        ipc_elem = doc.find(".//{http://www.epo.org/exchange}classifications-ipcr")
        ipc_texts = []
        if ipc_elem is not None:
            for cl in ipc_elem.findall("{http://www.epo.org/exchange}classification-ipcr"):
                text_elem = cl.find("{http://www.epo.org/exchange}text")
                if text_elem is not None and text_elem.text:
                    ipc_texts.append(text_elem.text.strip())
        record["ipc_classification"] = "; ".join(ipc_texts)

        records.append(record)
    return records


# --- Main logic ---
access_token = get_access_token()
print("Access token obtained.\n")

all_records = []
batch_size = 100
total_results = 500

for start in range(1, total_results + 1, batch_size):
    end = start + batch_size - 1
    print(f"Fetching results {start} to {end}...")
    
    url = "https://ops.epo.org/3.2/rest-services/published-data/search/biblio"
    params = {
        "q": "ti=plastic",
        "Range": f"{start}-{end}"
    }

    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/xml"
    }

    response = requests.get(url, params=params, headers=headers)
    
    # Handle rate limits or retry
    if response.status_code == 429:
        print("Rate limit reached, sleeping for 10 seconds...")
        time.sleep(10)
        continue
    elif response.status_code != 200:
        print(f"Error fetching batch {start}-{end}: {response.status_code}")
        continue

    batch_records = parse_xml_to_records(response.content)
    all_records.extend(batch_records)
    time.sleep(1)  # To avoid hitting API rate limits

# --- Create DataFrame ---
df = pd.DataFrame(all_records)
print(f"\nTotal results fetched: {len(df)}")
df.head()


Access token obtained.

Fetching results 1 to 100...
Fetching results 101 to 200...
Fetching results 201 to 300...
Fetching results 301 to 400...
Fetching results 401 to 500...

Total results fetched: 500


,publication_number,country,publication_date,priority_date,applicants,inventors,title,abstract,ipc_classification
0,PH22023050866U1,PH,20230721,20230707,"WANG ZHIMENG [CN]; WANG, Zhimeng","WANG ZHIMENG [CN]; WANG, Zhimeng",PLASTIC BOTTLE FOR GRANULAR LAUNDRY FRESHENER ...,,
1,KR20250041856A,KR,20250326,20230919,한국생산기술연구원,"김주혜,; 이소민",MICRO-PLASTIC COLLECTION LAUNDRY NET,세탁 중에 발생한 미세 플라스틱이 의류에 부착되는 것을 억제할 수 있는 미세 플라스...,D06F 95/ 00 A I
2,KR102781814B1,KR,20250318,20240911,(주)램코,박종팔,Waste plastic coating peeling system,"본 발명은 폐플라스틱에서 도장(코팅)된 표면층, 즉 도막을 효과적으로 제거할 수 있...",B08B 3/ 02 A I; B29B 13/ 0...
3,KR20250038181A,KR,20250319,20230911,주식회사 엘지화학,"최원종,; 박종건,; 임치훈,; 이미르,; 이준규",PREPARATION METHOD OF RECYCLED PLASTIC,본 발명은 폐플라스틱을 친환경적이고 우수한 품질로 재생하기 위한 재활용 플라스틱 제...,B29B 13/ 06 A I; B29B 17/ 0...
4,KR102783082B1,KR,20250319,20240710,주식회사 대경에스코,"심성훈,; 윤학상,; 박종욱",Continuous waste plastic pyrolysis apparatus w...,본 발명은 고품질의 열분해유 생성이 가능한 개별온도 제어 방식의 연속식 폐플라스틱의...,C10B 47/ 44 A I; C10B 53/ 0...


In [133]:
df.head(15)

,publication_number,country,publication_date,priority_date,applicants,inventors,title,abstract,ipc_classification
0,PH22023050866U1,PH,20230721,20230707,"WANG ZHIMENG [CN]; WANG, Zhimeng","WANG ZHIMENG [CN]; WANG, Zhimeng",PLASTIC BOTTLE FOR GRANULAR LAUNDRY FRESHENER ...,,
1,KR20250041856A,KR,20250326,20230919,한국생산기술연구원,"김주혜,; 이소민",MICRO-PLASTIC COLLECTION LAUNDRY NET,세탁 중에 발생한 미세 플라스틱이 의류에 부착되는 것을 억제할 수 있는 미세 플라스...,D06F 95/ 00 A I
2,KR102781814B1,KR,20250318,20240911,(주)램코,박종팔,Waste plastic coating peeling system,"본 발명은 폐플라스틱에서 도장(코팅)된 표면층, 즉 도막을 효과적으로 제거할 수 있...",B08B 3/ 02 A I; B29B 13/ 0...
3,KR20250038181A,KR,20250319,20230911,주식회사 엘지화학,"최원종,; 박종건,; 임치훈,; 이미르,; 이준규",PREPARATION METHOD OF RECYCLED PLASTIC,본 발명은 폐플라스틱을 친환경적이고 우수한 품질로 재생하기 위한 재활용 플라스틱 제...,B29B 13/ 06 A I; B29B 17/ 0...
4,KR102783082B1,KR,20250319,20240710,주식회사 대경에스코,"심성훈,; 윤학상,; 박종욱",Continuous waste plastic pyrolysis apparatus w...,본 발명은 고품질의 열분해유 생성이 가능한 개별온도 제어 방식의 연속식 폐플라스틱의...,C10B 47/ 44 A I; C10B 53/ 0...
5,WO2025065391A1,WO,20250403,20230927,GEELY HOLDING GROUP CO LTD [CN]; GEELY AUTOMOB...,YANG XIAOMENG [CN]; BAI FUJUN [CN]; WANG XUNJI...,PLASTIC COVER AND VEHICLE,A plastic cover and a vehicle. The plastic cov...,B29C 33/ 42 A I; B60R 13/ 0...
6,WO2025065117A1,WO,20250403,20230927,WENCO SA [CL]; WENCO S.A,"MUZZO SANTIAGO [CL]; MUZZO, Santiago",PLASTIC BOX,The present invention relates to a plastic box...,B65D 1/ 00 A I; B65D 1/ 3...
7,WO2025069016A1,WO,20250403,20230930,BADAN RISET DAN INOVASI NASIONAL BRIN [ID]; BA...,KISMANTO AGUS [ID]; RAKSODEWANTO ALFONSUS AGUS...,AUGER REACTOR FOR PROCESSING PLASTIC WASTE INT...,The pyrolysis process of plastic waste into li...,C10B 47/ 44 A I; C10B 53/ 0...
8,PH22023051628U1,PH,20231213,20231201,"CHUA ELISA G [PH]; CHUA, Elisa G","CHUA ELISA G [PH]; CHUA, Elisa G",PLASTIC CONTAINER WITH IMPROVED LOCKABLE HANDLE,,
9,MX395321B,MX,20250325,20120213,POLYTEX FIBERS LLC [US]; POLYTEX FIBERS LLC,BAZBAZ JACOBO [US]; BAZBAZ Jacobo,EASY OPEN WOVEN PLASTIC BAGS,A woven laminated plastic bag having an easy o...,B31B 50/ 81 A I; B32B 27/ 0...


ipc class extraction  

In [5]:
import os
import requests
import xml.etree.ElementTree as ET
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

def get_access_token():
    """Obtain an OAuth access token using credentials from .env."""
    token_url = "https://ops.epo.org/3.2/auth/accesstoken"
    response = requests.post(
        token_url,
        data={
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    response.raise_for_status()
    return response.json()["access_token"]

def format_classification_code(code):
    """
    The OPS endpoint expects the classification code to have a space after the first four characters.
    For example, if the user enters 'B31B50/00', we convert it to 'B31B 50/00'.
    """
    if " " not in code and len(code) > 4:
        return code[:4] + " " + code[4:]
    return code

def fetch_classification_data(user_code):
    """
    Fetch the CPC classification XML data for the given code.
    The user enters a code like 'B31B50/00' and we convert it to the proper format.
    """
    token = get_access_token()
    formatted_code = format_classification_code(user_code)  # e.g. "B31B50/00" -> "B31B 50/00"
    url = f"https://ops.epo.org/3.2/rest-services/classification/ipc/{formatted_code}?ancestors=&navigation=&depth="
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.content

def extract_ipc_meaning(xml_data, user_code):
    """
    Parse the XML and extract the classification meaning.
    We search for the <cpc:classification-item> that has a classification-symbol matching the user_code.
    Then we extract the text from the <cpc:class-title> (usually within <cpc:title-part> elements).
    """
    ns = {
        "ops": "http://ops.epo.org",
        "cpc": "http://www.epo.org/cpcexport",
        "cpcdef": "http://www.epo.org/cpcdefinition"
    }
    root = ET.fromstring(xml_data)
    meaning = None

    # Loop over all classification-item elements
    for item in root.findall(".//cpc:classification-item", ns):
        symbol_elem = item.find("cpc:classification-symbol", ns)
        if symbol_elem is not None and symbol_elem.text.strip() == user_code:
            # Found a matching classification item
            title_elem = item.find("cpc:class-title", ns)
            if title_elem is not None:
                # Collect all text parts from the title (if there are multiple <cpc:title-part> elements)
                parts = [tp.text.strip() for tp in title_elem.findall("cpc:title-part/cpc:text", ns) if tp.text]
                meaning = " ".join(parts)
            break
    return meaning

if __name__ == "__main__":
    # Prompt user for a 5-digit classification code, e.g. "B31B50/00"
    user_code = input("Enter a 5-digit CPC classification code (e.g. B31B50/00): ").strip()
    try:
        xml_response = fetch_classification_data(user_code)
    except requests.HTTPError as e:
        print("Error fetching classification data:", e)
        exit(1)
    
    meaning = extract_ipc_meaning(xml_response, user_code)
    if meaning:
        print(f"\nThe meaning for classification code {user_code} is:\n{meaning}")
    else:
        print(f"\nCould not find a matching classification meaning for code {user_code}.")


Error fetching classification data: 404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/classification/ipc/B31B%2050?ancestors=&navigation=&depth=

Could not find a matching classification meaning for code B31B50.


In [ ]:
import os
import requests
import xml.etree.ElementTree as ET
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

def get_access_token():
    """Obtain an OAuth access token using credentials from .env."""
    token_url = "https://ops.epo.org/3.2/auth/accesstoken"
    response = requests.post(
        token_url,
        data={
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    response.raise_for_status()
    return response.json()["access_token"]

def format_classification_code(code):
    """
    Insert a space after the first four characters.
    For example "B29B11" becomes "B29B 11".
    """
    if len(code) > 4 and " " not in code:
        return code[:4] + " " + code[4:]
    return code

def fetch_classification_data(class_code):
    """Fetch the CPC classification XML data for a given fixed classification code."""
    token = get_access_token()
    formatted_code = format_classification_code(class_code)  # e.g., "B29B11" -> "B29B 11"
    url = f"https://ops.epo.org/3.2/rest-services/classification/cpc/{formatted_code}?ancestors=&navigation=&depth="
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.content

def extract_ipc_meaning(xml_data, class_code):
    """
    Parse the XML and extract the classification meaning.
    We search for the <cpc:classification-item> that has a classification-symbol 
    matching the given class_code (after removing spaces) and then extract the CPC class-title text.
    """
    ns = {
        "ops": "http://ops.epo.org",
        "cpc": "http://www.epo.org/cpcexport",
        "cpcdef": "http://www.epo.org/cpcdefinition"
    }
    # Use errors="ignore" to drop invalid tokens
    xml_string = xml_data.decode('utf-8-sig', errors='ignore')
    
    # Print the beginning of the XML to check for any obvious issues
    print("Response excerpt:\n", xml_string[:150], "\n")
    
    try:
        root = ET.fromstring(xml_string)
    except ET.ParseError as err:
        print("XML parsing error:", err)
        return None

    meaning = None
    # Iterate over all classification items
    for item in root.findall(".//cpc:classification-item", ns):
        symbol_elem = item.find("cpc:classification-symbol", ns)
        if symbol_elem is not None:
            # Remove spaces from XML text and compare with user code.
            symbol_text = symbol_elem.text.replace(" ", "")
            if symbol_text == class_code:
                title_elem = item.find("cpc:class-title", ns)
                if title_elem is not None:
                    parts = [tp.text.strip() for tp in title_elem.findall("cpc:title-part/cpc:text", ns) if tp.text]
                    meaning = " ".join(parts)
                break
    return meaning

# Fixed classification code string (e.g., "B29B11")
classification_code = "B29B11"

# Fetch the XML data and extract the classification meaning
try:
    xml_response = fetch_classification_data(classification_code)
except Exception as e:
    print("Error fetching XML data:", e)
    xml_response = None

if xml_response:
    meaning = extract_ipc_meaning(xml_response, classification_code)
    if meaning:
        print(f"\nThe meaning for the CPC classification code {classification_code} is:\n{meaning}")
    else:
        print(f"\nCould not extract a valid meaning for the classification code {classification_code}.")
else:
    print("No XML response received.")


Response excerpt:
 <?xml-stylesheet type='text/xsl' href='../../../style/cpc.xsl' ?>{"code":404,"message":"Invalid or unknown class B29B11"} 

XML parsing error: not well-formed (invalid token): line 1, column 65

Could not extract a valid meaning for the classification code B29B11.


In [21]:
import os
import requests
from dotenv import load_dotenv
import json

# Load your .env credentials
load_dotenv()
client_id = os.getenv("CONSUMER_KEY")
client_secret = os.getenv("CONSUMER_SECRET")

# Get access token
def get_access_token():
    token_url = "https://ops.epo.org/3.2/auth/accesstoken"
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    response = requests.post(token_url, data=data, headers=headers)
    response.raise_for_status()
    return response.json()["access_token"]

def format_ipc_code(code: str) -> str:
    """
    Convert an IPC code like 'B29B11' or 'B29B1100' to 'B29B%2011/00'
    """
    section = code[:4]  # B29B
    subclass = code[4:6]  # 11
    group = code[6:8] if len(code) > 6 else "00"  # 00 if not provided
    return f"{section}%20{subclass}/{group}"


# Fetch IPC info for a fixed code
def fetch_ipc_info(ipc_code):
    access_token = get_access_token()
    formatted_code = ipc_code[:4] + " " + ipc_code[4:]  # e.g., B29B11 -> B29B 11
    url = f"https://ops.epo.org/3.2/rest-services/classification/cpc/{formatted_code}?ancestors=&navigation=&depth="
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"  # Only JSON!
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# Example IPC code
ipc_code = "B29B11"
ipc_code =format_ipc_code(ipc_code)

# Run and print
try:
    data = fetch_ipc_info(ipc_code)
    print(json.dumps(data, indent=2))
except Exception as e:
    print("Error:", e)


{
  "ops:world-patent-data": {
    "@xmlns": {
      "reg": "http://www.epo.org/register",
      "xlink": "http://www.w3.org/1999/xlink",
      "cpc": "http://www.epo.org/cpcexport",
      "cpcdef": "http://www.epo.org/cpcdefinition",
      "ops": "http://ops.epo.org"
    },
    "ops:classification-scheme": {
      "ops:cpc": {
        "ops:navigation": {
          "ops:prev": {
            "@xlink:href": "classification/cpc/B29B9/00",
            "$": "B29B9/00"
          },
          "ops:next": {
            "@xlink:href": "classification/cpc/B29B13/00",
            "$": "B29B13/00"
          }
        },
        "cpc:class-scheme": {
          "@scheme-type": "cpc",
          "@export-date": "2025-01-01",
          "cpc:classification-item": {
            "@level": "2",
            "@additional-only": "false",
            "@sort-key": "B",
            "@not-allocatable": "false",
            "@breakdown-code": "false",
            "@status": "published",
            "@link-file": "

register api point : offers applicant address and fresh data but covers only EP/WO patents

In [114]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# Load environment variables from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

def get_token():
    """Get OAuth access token using credentials from .env"""
    token_url = "https://ops.epo.org/3.2/auth/accesstoken"
    response = requests.post(
        token_url,
        data={
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    response.raise_for_status()
    return response.json()["access_token"]
# Function to call the OPS Register endpoint
def fetch_patent_data(query='applicant=Samsung'):
    token = get_token()
    url = "https://ops.epo.org/3.2/rest-services/register/search/biblio"

    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml"
    }

    params = {
        "q": query
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()

    return response.content

# Function to parse XML and extract data into DataFrame
def parse_patent_data(xml_data):
    root = ET.fromstring(xml_data)
    ns = {
        'reg': 'http://www.epo.org/register'
    }

    records = []
    for doc in root.findall('.//reg:register-document', ns):
        pub_number = doc.find('.//reg:publication-reference/reg:document-id/reg:doc-number', ns)
        status = doc.find('.//reg:ep-patent-statuses/reg:ep-patent-status', ns)
        states = doc.findall('.//reg:designation-of-states//reg:country', ns)
        applicant_country = doc.find('.//reg:applicants/reg:applicant/reg:addressbook/reg:address/reg:country', ns)

        records.append({
            "Publication Number": pub_number.text if pub_number is not None else None,
            "Status": status.text if status is not None else None,
            "Designated States": ', '.join([s.text for s in states]) if states else None,
            "Applicant Country": applicant_country.text if applicant_country is not None else None
        })

    df = pd.DataFrame(records)
    return df

# Usage
if __name__ == "__main__":
    xml_response = fetch_patent_data()
    register_df = parse_patent_data(xml_response)
    register_df.head()


In [115]:
register_df.head()

,Publication Number,Status,Designated States,Applicant Country
0,2025071385,The international publication has been made,"EP, AL, AT, BE, BG, CH, CY, CZ, DE, DK, EE, ES...",KR
1,2025071384,The international publication has been made,"EP, AL, AT, BE, BG, CH, CY, CZ, DE, DK, EE, ES...",KR
2,2025071382,The international publication has been made,"EP, AL, AT, BE, BG, CH, CY, CZ, DE, DK, EE, ES...",KR
3,2025071381,The international publication has been made,"EP, AL, AT, BE, BG, CH, CY, CZ, DE, DK, EE, ES...",KR
4,2025071378,The international publication has been made,"EP, AL, AT, BE, BG, CH, CY, CZ, DE, DK, EE, ES...",KR


In [126]:
import os
import time
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from dotenv import load_dotenv

# --- Load environment variables ---
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

def get_token():
    """Get OAuth access token using credentials from .env"""
    token_url = "https://ops.epo.org/3.2/auth/accesstoken"
    response = requests.post(
        token_url,
        data={
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    response.raise_for_status()
    return response.json()["access_token"]

# --- Fetch patent data from the endpoint ---
def fetch_patent_data(query="ti=plastic", range_str="1-20"):
    token = get_access_token()
    url = "https://ops.epo.org/3.2/rest-services/register/search/biblio"
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml"
    }
    params = {
        "q": query,
        "Range": range_str
    }
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.content

# --- Parse XML into records ---
def parse_patent_xml(xml_data):
    ns = {
        "reg": "http://www.epo.org/register"
    }
    records = []
    try:
        root = ET.fromstring(xml_data)
    except ET.ParseError as err:
        print("XML parsing error:", err)
        return records

    # Loop over each register-document element
    for doc in root.findall('.//reg:register-document', ns):
        # Extract Publication Number as country+doc-number+kind with no spaces
        pub_ref = doc.find('.//reg:publication-reference/reg:document-id', ns)
        pub_country = pub_ref.find("reg:country", ns).text.strip() if pub_ref is not None and pub_ref.find("reg:country", ns) is not None else ""
        pub_docnum = pub_ref.find("reg:doc-number", ns).text.strip() if pub_ref is not None and pub_ref.find("reg:doc-number", ns) is not None else ""
        pub_kind = pub_ref.find("reg:kind", ns).text.strip() if pub_ref is not None and pub_ref.find("reg:kind", ns) is not None else ""
        publication_number = f"{pub_country}{pub_docnum}{pub_kind}"
        
        # Extract Status from <reg:ep-patent-status>
        status_elem = doc.find('.//reg:ep-patent-statuses/reg:ep-patent-status', ns)
        status = status_elem.text.strip() if status_elem is not None and status_elem.text else ""

        # Extract Designated States (all designated countries within <reg:designation-of-states>)
        state_elems = doc.findall('.//reg:designation-of-states//reg:country', ns)
        designated_states = "; ".join([s.text.strip() for s in state_elems if s.text])
        
        # Extract Applicant Country from first applicant's address
        applicant_country_elem = doc.find('.//reg:applicants/reg:applicant/reg:addressbook/reg:address/reg:country', ns)
        applicant_country = applicant_country_elem.text.strip() if applicant_country_elem is not None and applicant_country_elem.text else ""
        
        # Build record dictionary
        record = {
            "Publication Number": publication_number,
            "Status": status,
            "Designated States": designated_states,
            "Applicant Country": applicant_country
        }
        records.append(record)
    return records

# --- Main ---
if __name__ == "__main__":
    # Fetch data using the endpoint. Here, we're fetching the first 2 results.
    # To get a larger batch, modify the Range, for example "1-100" or use pagination.
    xml_data = fetch_patent_data(query="ti=plastic", range_str="1-20")
    records = parse_patent_xml(xml_data)
    
    # Convert records into a DataFrame
    register_df = pd.DataFrame(records)
    register_df.head()


In [129]:
register_df.tail()

,Publication Number,Status,Designated States,Applicant Country
15,WO2025062273A1,The international publication has been made,EP; AL; AT; BE; BG; CH; CY; CZ; DE; DK; EE; ES...,IT
16,WO2025062239A1,The international publication has been made,EP; AL; AT; BE; BG; CH; CY; CZ; DE; DK; EE; ES...,IE
17,WO2025062238A1,The international publication has been made,EP; AL; AT; BE; BG; CH; CY; CZ; DE; DK; EE; ES...,IE
18,WO2025061730A1,The international publication has been made,EP; AL; AT; BE; BG; CH; CY; CZ; DE; DK; EE; ES...,IT
19,WO2025064546A1,The international publication has been made,EP; AL; AT; BE; BG; CH; CY; CZ; DE; DK; EE; ES...,US


family members jusrisdictions


In [24]:
import requests

# Your credentials
consumer_key = "0xdYfz2zFpkbpsgNLnM75LIEt8ayfjzyTzeL00g45m9814np"
consumer_secret = "TV8jBWPaGwV3iqHkZuKnVmMKlLOYxpQgG9DcdE7Zbij5ROhLWsUfNiqUzklQAylG"

# Get access token
token_url = "https://ops.epo.org/3.2/auth/accesstoken"
token_response = requests.post(
    token_url,
    data={
        "grant_type": "client_credentials",
        "client_id": consumer_key,
        "client_secret": consumer_secret
    },
    headers={"Content-Type": "application/x-www-form-urlencoded"}
)

access_token = token_response.json()["access_token"]

# Make API request (fixed URL by removing %20)
api_url = "https://ops.epo.org/3.2/rest-services/family/publication/docdb/US2014006294A1"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json"
}

response = requests.get(api_url, headers=headers)
family_data = response.json()

# Extract distinct jurisdictions
def get_family_jurisdictions(data):
    jurisdictions = set()
    
    # Get all family members
    family_members = data['ops:world-patent-data']['ops:patent-family']['ops:family-member']
    
    # Handle case where there's only one member (not a list)
    if not isinstance(family_members, list):
        family_members = [family_members]
    
    for member in family_members:
        # Get publication references
        pub_refs = member['publication-reference']['document-id']
        if not isinstance(pub_refs, list):
            pub_refs = [pub_refs]
            
        for ref in pub_refs:
            if ref['@document-id-type'] == 'docdb':
                jurisdictions.add(ref['country']['$'])
    
    return sorted(jurisdictions)

# Get and print jurisdictions
jurisdictions = get_family_jurisdictions(family_data)
print("Patent family jurisdictions:", jurisdictions)

Patent family jurisdictions: ['JP', 'KR', 'US']


In [8]:
import requests

# Configuration
PATENT_NUMBERS = [
    "CN103164540A",
    "CN105930942A",
    "CN112508743A"
]
OUTPUT_FORMAT = "dict"  # or "dict" for dictionary output

# OPS API credentials
CONSUMER_KEY = "0xdYfz2zFpkbpsgNLnM75LIEt8ayfjzyTzeL00g45m9814np"
CONSUMER_SECRET = "TV8jBWPaGwV3iqHkZuKnVmMKlLOYxpQgG9DcdE7Zbij5ROhLWsUfNiqUzklQAylG"

def get_access_token():
    """Get OAuth access token"""
    token_url = "https://ops.epo.org/3.2/auth/accesstoken"
    response = requests.post(
        token_url,
        data={
            "grant_type": "client_credentials",
            "client_id": CONSUMER_KEY,
            "client_secret": CONSUMER_SECRET
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    response.raise_for_status()
    return response.json()["access_token"]

def get_family_jurisdictions(patent_number, access_token):
    """Get distinct jurisdictions for a patent family"""
    api_url = f"https://ops.epo.org/3.2/rest-services/family/publication/docdb/{patent_number}"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }
    
    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        jurisdictions = set()
        family_members = data['ops:world-patent-data']['ops:patent-family']['ops:family-member']
        
        if not isinstance(family_members, list):
            family_members = [family_members]
            
        for member in family_members:
            pub_refs = member['publication-reference']['document-id']
            if not isinstance(pub_refs, list):
                pub_refs = [pub_refs]
                
            for ref in pub_refs:
                if ref['@document-id-type'] == 'docdb':
                    jurisdictions.add(ref['country']['$'])
        
        return sorted(jurisdictions)
    
    except Exception as e:
        print(f"Error processing {patent_number}: {str(e)}")
        return None

def main():
    access_token = get_access_token()
    results = {}
    
    for patent_number in PATENT_NUMBERS:
        jurisdictions = get_family_jurisdictions(patent_number, access_token)
        if jurisdictions:
            results[patent_number] = jurisdictions
    
    if OUTPUT_FORMAT == "list":
        # Print as list of tuples
        for patent, jurisdictions in results.items():
            print(f"{patent}: {', '.join(jurisdictions)}")
    else:
        # Return as dictionary
        print(results)

if __name__ == "__main__":
    main()

{'CN103164540A': ['CN'], 'CN105930942A': ['CN'], 'CN112508743A': ['CN', 'US', 'WO']}


In [24]:
import pandas as pd
df = pd.read_csv('search-results-epo.csv',delimiter=';', skiprows=7)

In [25]:
df= df.rename(columns={'Numéro de publication': 'Publication number'})

In [26]:
df.head(1)

,No,Titre,Inventeurs,Demandeurs,Publication number,Priorité la plus ancienne,CIB,CPC,Date de publication,Publication la plus ancienne,Numéro de famille,Unnamed: 11
0,1,Plug-in hybrid vehicle with fast energy storage,BENDER DONALD A [US] \r\nDESHMANE ATUL [US] \r...,AFS TRINITY POWER CORP [US],US2006250902A1,2005-05-05,H04B1/20,B60K6/20 (KR) \r\nB60K6/28 (KR) \r\nB60K6/30 (...,2006-11-09,2006-11-09,37397092,NaN


In [27]:
df[['first publication number', 'second publication number']] = df['Publication number'].str.split(' ' , n=1 , expand=True)

In [28]:
df.head(2)

,No,Titre,Inventeurs,Demandeurs,Publication number,Priorité la plus ancienne,CIB,CPC,Date de publication,Publication la plus ancienne,Numéro de famille,Unnamed: 11,first publication number,second publication number
0,1,Plug-in hybrid vehicle with fast energy storage,BENDER DONALD A [US] \r\nDESHMANE ATUL [US] \r...,AFS TRINITY POWER CORP [US],US2006250902A1,2005-05-05,H04B1/20,B60K6/20 (KR) \r\nB60K6/28 (KR) \r\nB60K6/30 (...,2006-11-09,2006-11-09,37397092,NaN,US2006250902A1,None
1,2,Vehicle charging system capable of hydrogen an...,HONG SEONG HO,HOGREENAIR CO LTD [KR],KR102511398B1,2022-09-20,B60L3/00 \r\nB60L53/16 \r\nB60L53/51 \r\nB60L5...,Y02T10/70 (EP) \r\nY02T10/7072 (EP),2023-03-20,2023-03-20,85796445,NaN,KR102511398B1,None


In [64]:
df.tail(2)

,No,Titre,Inventeurs,Demandeurs,Publication number,Priorité la plus ancienne,CIB,CPC,Date de publication,Publication la plus ancienne,Numéro de famille,Unnamed: 11,first publication number,second publication number
498,499,Hydrogen fuel cell vehicle power system suppor...,LI KAIHAN \r\nHAO YIGUO \r\nHE YU \r\nAO XIAOPING,WUHAN GROVE HYDROGEN AUTOMOBILE CO LTD,CN110722990A \r\nCN110722990B,2019-10-31,B60L3/00 \r\nB60L58/12 \r\nB60L58/30 \r\nH01M1...,B60L3/0053 (CN) \r\nB60L58/12 (CN) \r\nB60L58/...,2020-01-24 \r\n2023-06-30,2020-01-24,69222658,NaN,CN110722990A,\r\nCN110722990B
499,500,"Fuel cell reactor start-stop control method, d...",WU JINLIN \r\nMA QIUXIANG \r\nTAN JIRONG \r\nW...,DONGFENG LIUZHOU MOTOR CO LTD,CN119099437A,2024-09-30,B60L58/30 \r\nB60L58/31,B60L58/30 (CN) \r\nB60L58/31 (CN),2024-12-10,2024-12-10,93720833,NaN,CN119099437A,None


In [72]:
import pandas as pd
import requests
import time
from urllib.parse import quote
from typing import List, Dict, Optional

class OPSBatchProcessor:
    def __init__(self, consumer_key: str, consumer_secret: str):
        self.consumer_key = consumer_key.strip()
        self.consumer_secret = consumer_secret.strip()
        self.token_url = "https://ops.epo.org/3.2/auth/accesstoken"
        self.base_url = "https://ops.epo.org/3.2/rest-services"
        self.token = None
        self.token_expiry = 0
        self.batch_size = 50
        self.request_delay = 1.2  # 50 requests/minute
        
    def _get_token(self) -> str:
        """Get or refresh token with enhanced validation"""
        if self.token and time.time() < self.token_expiry:
            return self.token
            
        try:
            response = requests.post(
                self.token_url,
                data={
                    "grant_type": "client_credentials",
                    "client_id": self.consumer_key,
                    "client_secret": self.consumer_secret
                },
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                timeout=15
            )
            response.raise_for_status()
            self.token = response.json()["access_token"]
            self.token_expiry = time.time() + 3500  # 58.3 minutes
            return self.token
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 403:
                raise ValueError(
                    "Authentication failed. Please verify:\n"
                    "1. Your credentials are correct\n"
                    "2. Your account is activated\n"
                    "3. You have proper subscription for OPS services"
                )
            raise

    def _validate_patent_number(self, patent: str) -> bool:
        """Basic validation for patent number format"""
        if not patent or len(patent) < 4:
            return False
        return True

    def _process_patent(self, patent: str) -> Optional[List[str]]:
        """Process single patent with comprehensive error handling"""
        if not self._validate_patent_number(patent):
            print(f"Invalid patent number format: {patent}")
            return None
            
        try:
            token = self._get_token()
            response = requests.get(
                f"{self.base_url}/family/publication/docdb/{quote(patent)}",
                headers={
                    "Authorization": f"Bearer {token}",
                    "Accept": "application/json"
                },
                timeout=15
            )
            
            if response.status_code == 403:
                print(f"Access forbidden for patent {patent} - may not exist in OPS")
                return None
            if response.status_code == 404:
                print(f"Patent {patent} not found in OPS database")
                return None
                
            response.raise_for_status()
            return self._extract_jurisdictions(response.json())
            
        except Exception as e:
            print(f"Error processing {patent}: {str(e)}")
            return None

    def _extract_jurisdictions(self, data: dict) -> Optional[List[str]]:
        """Extract jurisdictions from API response"""
        try:
            jurisdictions = set()
            family = data.get('ops:world-patent-data', {}).get('ops:patent-family', {})
            
            # Handle both single member and list cases
            members = family.get('ops:family-member', [])
            if isinstance(members, dict):
                members = [members]
                
            for member in members:
                docs = member.get('publication-reference', {}).get('document-id', [])
                if isinstance(docs, dict):
                    docs = [docs]
                    
                for doc in docs:
                    if doc.get('@document-id-type') == 'docdb':
                        if country := doc.get('country', {}).get('$'):
                            jurisdictions.add(country)
            
            return sorted(jurisdictions) if jurisdictions else None
            
        except Exception as e:
            print(f"Error parsing response: {str(e)}")
            return None

    def process_dataframe(self, df: pd.DataFrame, patent_col: str) -> pd.DataFrame:
        """Process DataFrame with enhanced error handling"""
        if patent_col not in df.columns:
            raise ValueError(f"Column '{patent_col}' not found")
            
        result_df = df.copy()
        patents = result_df[patent_col].tolist()
        total = len(patents)
        results = {}
        
        for i in range(0, total, self.batch_size):
            batch = patents[i:i + self.batch_size]
            print(f"\nProcessing batch {i//self.batch_size + 1}/{(total-1)//self.batch_size + 1}")
            
            for patent in batch:
                results[patent] = self._process_patent(patent)
                time.sleep(self.request_delay)
                
            # Pause between batches
            if i + self.batch_size < total:
                print("Pausing between batches...")
                time.sleep(1)
        
        result_df['family_jurisdictions'] = result_df[patent_col].map(results)
        return result_df

# Usage Example
if __name__ == "__main__":
    # Initialize with your credentials
    processor = OPSBatchProcessor(
        consumer_key="0xdYfz2zFpkbpsgNLnM75LIEt8ayfjzyTzeL00g45m9814np",
        consumer_secret="TV8jBWPaGwV3iqHkZuKnVmMKlLOYxpQgG9DcdE7Zbij5ROhLWsUfNiqUzklQAylG"
    )
    
    # Test credentials first
    try:
        test_token = processor._get_token()
        print("Authentication successful")
    except Exception as e:
        print(f"Authentication failed: {str(e)}")
        exit()

    # Sample DataFrame (replace with your data)
    # df = pd.DataFrame({
    #     'first_publication_number': [
    #         'US2006250902A1', 'KR102511391B1', 'EP2840123A1', 
    #         'INVALID_PATENT', 'US2022140365A1'
    #     ],
    #     'title': ['A', 'B', 'C', 'D', 'E']
    # })
    
    # Process and save results
    try:
        result_df = processor.process_dataframe(df, 'first publication number')
        print("\nFinal Results:")
        print(result_df[['first publication number', 'family_jurisdictions']])
        
        result_df.to_csv('patent_jurisdictions.csv', index=False)
    except Exception as e:
        print(f"Processing failed: {str(e)}")

Authentication successful

Processing batch 1/6
Pausing between batches...

Processing batch 2/6
Pausing between batches...

Processing batch 3/6
Pausing between batches...

Processing batch 4/6
Pausing between batches...

Processing batch 5/6
Pausing between batches...

Processing batch 6/6

Final Results:
    first publication number                      family_jurisdictions
0             US2006250902A1      [AU, CA, CN, EP, HK, JP, KR, US, WO]
1              KR102511398B1                                      [KR]
2              KR102511391B1                                      [KR]
3             KR20230163874A                                      [KR]
4                 GB2631101A                                  [GB, WO]
..                       ...                                       ...
295             CN118211793A                                      [CN]
296             CN216331451U                                      [CN]
297           US2023331108A1                        

In [68]:
df=df.head(300)

In [ ]:
df

In [ ]:
import pandas as pd
import requests
import time
from urllib.parse import quote
from typing import List, Optional

class OPSHybridProcessor:
    def __init__(self, consumer_key: str, consumer_secret: str):
        self.consumer_key = consumer_key.strip()
        self.consumer_secret = consumer_secret.strip()
        self.token_url = "https://ops.epo.org/3.2/auth/accesstoken"
        self.base_url = "https://ops.epo.org/3.2/rest-services"
        self.token = None
        self.request_delay = 1.2  # 50 requests/minute
        
    def _get_token(self) -> str:
        """Token management from working version"""
        try:
            response = requests.post(
                self.token_url,
                data={
                    "grant_type": "client_credentials",
                    "client_id": self.consumer_key,
                    "client_secret": self.consumer_secret
                },
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                timeout=10
            )
            response.raise_for_status()
            return response.json()["access_token"]
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 403:
                raise ValueError("Authentication failed - check credentials")
            raise

    def get_jurisdictions(self, patent_number: str) -> Optional[List[str]]:
        """Working version's request handling"""
        if not self.token:
            self.token = self._get_token()
            
        try:
            response = requests.get(
                f"{self.base_url}/family/publication/docdb/{quote(patent_number)}",
                headers={
                    "Authorization": f"Bearer {self.token}",
                    "Accept": "application/json"
                },
                timeout=15
            )
            response.raise_for_status()
            return self._parse_response(response.json())
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 401:  # Token expired
                self.token = self._get_token()
                return self.get_jurisdictions(patent_number)
            print(f"Error processing {patent_number}: {str(e)}")
            return None
        except Exception as e:
            print(f"Error processing {patent_number}: {str(e)}")
            return None

    def _parse_response(self, data: dict) -> Optional[List[str]]:
        """Response parsing from working version"""
        jurisdictions = set()
        members = data.get('ops:world-patent-data', {}).get('ops:patent-family', {}).get('ops:family-member', [])
        
        if isinstance(members, dict):
            members = [members]
            
        for member in members:
            docs = member.get('publication-reference', {}).get('document-id', [])
            if isinstance(docs, dict):
                docs = [docs]
                
            for doc in docs:
                if doc.get('@document-id-type') == 'docdb':
                    if country := doc.get('country', {}).get('$'):
                        jurisdictions.add(country)
        
        return sorted(jurisdictions) if jurisdictions else None

    def process_dataframe(self, df: pd.DataFrame, patent_col: str) -> pd.DataFrame:
        """Batch processing with working version's logic"""
        result_df = df.copy()
        patents = result_df[patent_col].tolist()
        
        print(f"Processing {len(patents)} patents...")
        
        result_df['family_jurisdictions'] = [
            self._process_with_delay(patent) 
            for patent in patents
        ]
        
        return result_df

    def _process_with_delay(self, patent: str) -> Optional[List[str]]:
        """Add rate limiting to working version"""
        result = self.get_jurisdictions(patent)
        time.sleep(self.request_delay)
        return result

# Usage Example
if __name__ == "__main__":
    # Initialize with your credentials
    processor = OPSHybridProcessor(
        consumer_key="0xdYfz2zFpkbpsgNLnM75LIEt8ayfjzyTzeL00g45m9814np",
        consumer_secret="TV8jBWPaGwV3iqHkZuKnVmMKlLOYxpQgG9DcdE7Zbij5ROhLWsUfNiqUzklQAylG"
    )
    
    # Test credentials
    try:
        processor._get_token()
        print("Authentication successful")
    except Exception as e:
        print(f"Authentication failed: {str(e)}")
        exit()


    # Process DataFrame
    result_df = processor.process_dataframe(df, 'first publication number')
    print(result_df[['first publication number', 'family_jurisdictions']])

Authentication successful
Processing 300 patents...
    first publication number                      family_jurisdictions
0             US2006250902A1      [AU, CA, CN, EP, HK, JP, KR, US, WO]
1              KR102511398B1                                      [KR]
2              KR102511391B1                                      [KR]
3             KR20230163874A                                      [KR]
4                 GB2631101A                                  [GB, WO]
..                       ...                                       ...
295             CN118211793A                                      [CN]
296             CN216331451U                                      [CN]
297           US2023331108A1                                      [US]
298              US6326765B1  [AU, CA, CN, EP, JP, KR, MX, TW, US, WO]
299             CN115566713A                                      [CN]

[300 rows x 2 columns]


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   No                            300 non-null    int64  
 1   Titre                         300 non-null    object 
 2   Inventeurs                    298 non-null    object 
 3   Demandeurs                    297 non-null    object 
 4   Publication number            300 non-null    object 
 5   Priorité la plus ancienne     300 non-null    object 
 6   CIB                           299 non-null    object 
 7   CPC                           262 non-null    object 
 8   Date de publication           300 non-null    object 
 9   Publication la plus ancienne  300 non-null    object 
 10  Numéro de famille             300 non-null    int64  
 11  Unnamed: 11                   0 non-null      float64
 12  first publication number      300 non-null    object 
 13  secon

In [74]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load the .env file

# Test if variables are loaded
print("Consumer Key:", os.getenv("EPO_CONSUMER_KEY"))
print("Consumer Secret:", bool(os.getenv("EPO_CONSUMER_SECRET")))  # Shows True/False without revealing secret

Consumer Key: 0xdYfz2zFpkbpsgNLnM75LIEt8ayfjzyTzeL00g45m9814np
Consumer Secret: True


In [22]:
import os
import requests
import time
from urllib.parse import quote
import pandas as pd
from dotenv import load_dotenv
import xml.etree.ElementTree as ET

# Constants for API endpoints
TOKEN_URL = "https://ops.epo.org/3.2/auth/accesstoken"
BASE_URL = "https://ops.epo.org/3.2/rest-services"

# Global token cache variables
TOKEN = None
TOKEN_EXPIRY = 0

# Load credentials from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY").strip()
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET").strip()

def get_access_token() -> str:
    """
    Get or refresh the OAuth access token.
    """
    global TOKEN, TOKEN_EXPIRY
    if TOKEN and time.time() < TOKEN_EXPIRY:
        return TOKEN
    data = {
        "grant_type": "client_credentials",
        "client_id": CONSUMER_KEY,
        "client_secret": CONSUMER_SECRET
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    response = requests.post(TOKEN_URL, data=data, headers=headers, timeout=15)
    response.raise_for_status()
    TOKEN = response.json()["access_token"]
    TOKEN_EXPIRY = time.time() + 3500  # about 58 minutes validity
    return TOKEN

def parse_english_abstract(xml_content: str) -> str:
    """
    Parse XML content and extract the English abstract.
    
    Expected structure:
      <abstract lang="en">
         <p>Text of abstract...</p>
      </abstract>
    """
    try:
        root = ET.fromstring(xml_content)
        # Define the default namespace as used in the XML
        ns = {'default': 'http://www.epo.org/exchange'}
        # Look for <abstract> elements in the default namespace
        abstracts = root.findall('.//default:abstract', ns)
        for abstract in abstracts:
            # Check the language attribute from the XML namespace for lang (xml:lang)
            if abstract.attrib.get('{http://www.w3.org/XML/1998/namespace}lang') == 'en':
                # Join text from all child <p> elements
                return ' '.join(p.text.strip() for p in abstract.findall('default:p', ns) if p.text)
        return None
    except ET.ParseError as e:
        print(f"XML parsing error: {e}")
        return None

def get_english_abstract(publication_number: str) -> str:
    """
    Retrieve the English abstract for a given publication number using the OPS abstract endpoint.
    
    For example, for publication_number = 'KR102511398B1',
    the endpoint URL becomes:
      https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/KR102511398B1/abstract
    """
    try:
        token = get_access_token()
        url = f"{BASE_URL}/published-data/publication/docdb/{quote(publication_number)}/abstract"
        headers = {
            "Authorization": f"Bearer {token}",
            "Accept": "application/xml"
        }
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code in (403, 404):
            print(f"Patent {publication_number} - Access forbidden or not found ({response.status_code})")
            return None

        response.raise_for_status()
        # Uncomment the next line to print raw XML during debugging
        # print(response.text[:300])
        return parse_english_abstract(response.text)
    except Exception as e:
        print(f"Failed to retrieve abstract for {publication_number}: {e}")
        return None

def add_abstracts_to_dataframe(df: pd.DataFrame, patent_col: str) -> pd.DataFrame:
    """
    Enrich the DataFrame by adding an 'english_abstract' column.
    Each row will query the OPS API for its abstract based on the publication number.
    """
    if patent_col not in df.columns:
        raise ValueError(f"Column '{patent_col}' not found in DataFrame")
    
    result_df = df.copy()
    abstracts = []
    total = len(result_df)
    for idx, pub_number in enumerate(result_df[patent_col]):
        print(f"Processing {idx+1}/{total}: {pub_number}")
        abstract = get_english_abstract(pub_number)
        abstracts.append(abstract)
        # Delay to respect OPS rate limits
        time.sleep(1.2)
    result_df['english_abstract'] = abstracts
    return result_df

# ======= USAGE EXAMPLE =======
if __name__ == "__main__":
    # Sample DataFrame; replace with your actual DataFrame
    # df = pd.DataFrame({
    #     'first publication number': [
    #         'KR102511398B1',   # Example from your XML
    #         'EP2840123A1', 
    #         'US2006250902A1'
    #     ],
    #     'title': ['Title A', 'Title B', 'Title C']
    # })
    
    # Process the DataFrame to add English abstracts
    enriched_df = add_abstracts_to_dataframe(df, 'first publication number')
    
    # Display a preview of the results
    enriched_df[['first publication number', 'english_abstract']]
    
    # Optionally, save the results to CSV for use by your frontend
    enriched_df.to_csv("patents_with_abstracts.csv", index=False)


NameError: name 'df' is not defined

In [81]:
df.tail()

,publication_number,title,applicants,ipc,family_size,no
95,2025071112,,SAMSUNG ELECTRONICS CO LTD [KR]; SAMSUNG ELECT...,H04W 8/ 04 A I; H04W 48/ 1...,1,96
96,2025072780,,VISA INT SERVICE ASSOCIATION [US]; VISA INTERN...,G06Q 20/ 02 A I; G06Q 20/ 0...,1,97
97,2025067570,,ESSENLIX BIOTECHNOLOGY SHANGHAI CO LTD [CN]; 上...,G02B 21/ 00 A I; G02B 21/ 3...,2,98
98,2025072871,,MEMORIAL SLOAN KETTERING CANCER CENTER [US]; M...,A61K 38/ 19 A I; C07K 16/ 2...,1,99
99,2025072825,,SUN WORLD INT LLC [US]; SUN WORLD INTERNATIONA...,C12N 5/ 14 A I; C12N 15/ 8...,2,100


claude

flask api for epo patent data integration 

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
from epo_collector import EPOPatentCollector
import json
import os

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Initialize the EPO patent collector
collector = EPOPatentCollector()

@app.route('/api/search', methods=['POST'])
def search_patents():
    """
    API endpoint to search for patents based on user input.
    
    Expected JSON body:
    {
        "keywords": ["term1", "term2"] or "term",
        "max_results": 500,
        "date_range": "2018-01-01 TO 2023-12-31",
        "applicant": "Company Name",
        "classification": "G06F"
    }
    
    Returns:
        JSON response with patent data and status
    """
    try:
        # Get request data
        data = request.json
        
        if not data or 'keywords' not in data:
            return jsonify({
                'status': 'error',
                'message': 'Keywords are required'
            }), 400
        
        # Extract parameters
        keywords = data.get('keywords')
        max_results = data.get('max_results', 100)
        date_range = data.get('date_range')
        applicant = data.get('applicant')
        classification = data.get('classification')
        
        # Validate max_results
        try:
            max_results = int(max_results)
            if max_results <= 0 or max_results > 500:
                max_results = 100
        except (ValueError, TypeError):
            max_results = 100
        
        # Collect patent data
        df = collector.collect_patent_data(
            keywords=keywords,
            max_results=max_results,
            date_range=date_range,
            applicant=applicant,
            classification=classification
        )
        
        if df.empty:
            return jsonify({
                'status': 'warning',
                'message': 'No results found',
                'data': []
            })
        
        # Convert dataframe to list of dictionaries for JSON response
        patents = df.fillna('').to_dict(orient='records')
        
        # Return success response
        return jsonify({
            'status': 'success',
            'message': f'Found {len(patents)} patents',
            'data': patents
        })
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': f'An error occurred: {str(e)}'
        }), 500

@app.route('/api/export', methods=['POST'])
def export_patents():
    """
    API endpoint to export patent data as CSV or JSON.
    
    Expected JSON body:
    {
        "data": [...],  # Patent data from search results
        "format": "csv" or "json"
    }
    
    Returns:
        JSON response with file path or error
    """
    try:
        data = request.json
        
        if not data or 'data' not in data:
            return jsonify({
                'status': 'error',
                'message': 'No data provided'
            }), 400
        
        export_format = data.get('format', 'csv').lower()
        patent_data = data.get('data', [])
        
        if not patent_data:
            return jsonify({
                'status': 'warning',
                'message': 'No data to export'
            })
        
        # Create dataframe from patent data
        df = pd.DataFrame(patent_data)
        
        # Create exports directory if it doesn't exist
        os.makedirs('exports', exist_ok=True)
        
        # Generate filename
        filename = f"patent_export_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"
        
        if export_format == 'csv':
            filepath = f"exports/{filename}.csv"
            df.to_csv(filepath, index=False)
        else:
            filepath = f"exports/{filename}.json"
            df.to_json(filepath, orient='records', indent=2)
        
        return jsonify({
            'status': 'success',
            'message': f'Data exported as {export_format.upper()}',
            'filepath': filepath
        })
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': f'Export failed: {str(e)}'
        }), 500

@app.route('/api/analyze', methods=['POST'])
def analyze_patents():
    """
    API endpoint to perform basic analysis on patent data.
    
    Expected JSON body:
    {
        "data": [...]  # Patent data from search results
    }
    
    Returns:
        JSON response with analysis results
    """
    try:
        data = request.json
        
        if not data or 'data' not in data:
            return jsonify({
                'status': 'error',
                'message': 'No data provided'
            }), 400
        
        patent_data = data.get('data', [])
        
        if not patent_data:
            return jsonify({
                'status': 'warning',
                'message': 'No data to analyze'
            })
        
        # Create dataframe from patent data
        df = pd.DataFrame(patent_data)
        
        # Perform basic analysis
        analysis = {}
        
        # Publication trend by year
        if 'Publication date' in df.columns:
            df['Publication year'] = df['Publication date'].str[:4]
            pub_trend = df['Publication year'].value_counts().sort_index().to_dict()
            analysis['publication_trend'] = pub_trend
        
        # Top applicants
        if 'Applicants' in df.columns:
            # Split applicants by semicolon and count occurrences
            all_applicants = []
            for applicants in df['Applicants'].dropna():
                all_applicants.extend([app.strip() for app in applicants.split(';')])
            
            top_applicants = pd.Series(all_applicants).value_counts().head(10).to_dict()
            analysis['top_applicants'] = top_applicants
        
        # Technology distribution (based on IPC)
        if 'IPC' in df.columns:
            # Get main IPC class (first character)
            df['IPC_main'] = df['IPC'].str[:1]
            ipc_dist = df['IPC_main'].value_counts().to_dict()
            analysis['ipc_distribution'] = ipc_dist
        
        return jsonify({
            'status': 'success',
            'analysis': analysis
        })
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': f'Analysis failed: {str(e)}'
        }), 500

if __name__ == '__main__':
    app.run(debug=True, port=5000)

react frontend fro patent search ui 

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
from epo_collector import EPOPatentCollector
import json
import os

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Initialize the EPO patent collector
collector = EPOPatentCollector()

@app.route('/api/search', methods=['POST'])
def search_patents():
    """
    API endpoint to search for patents based on user input.
    
    Expected JSON body:
    {
        "keywords": ["term1", "term2"] or "term",
        "max_results": 500,
        "date_range": "2018-01-01 TO 2023-12-31",
        "applicant": "Company Name",
        "classification": "G06F"
    }
    
    Returns:
        JSON response with patent data and status
    """
    try:
        # Get request data
        data = request.json
        
        if not data or 'keywords' not in data:
            return jsonify({
                'status': 'error',
                'message': 'Keywords are required'
            }), 400
        
        # Extract parameters
        keywords = data.get('keywords')
        max_results = data.get('max_results', 100)
        date_range = data.get('date_range')
        applicant = data.get('applicant')
        classification = data.get('classification')
        
        # Validate max_results
        try:
            max_results = int(max_results)
            if max_results <= 0 or max_results > 500:
                max_results = 100
        except (ValueError, TypeError):
            max_results = 100
        
        # Collect patent data
        df = collector.collect_patent_data(
            keywords=keywords,
            max_results=max_results,
            date_range=date_range,
            applicant=applicant,
            classification=classification
        )
        
        if df.empty:
            return jsonify({
                'status': 'warning',
                'message': 'No results found',
                'data': []
            })
        
        # Convert dataframe to list of dictionaries for JSON response
        patents = df.fillna('').to_dict(orient='records')
        
        # Return success response
        return jsonify({
            'status': 'success',
            'message': f'Found {len(patents)} patents',
            'data': patents
        })
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': f'An error occurred: {str(e)}'
        }), 500

@app.route('/api/export', methods=['POST'])
def export_patents():
    """
    API endpoint to export patent data as CSV or JSON.
    
    Expected JSON body:
    {
        "data": [...],  # Patent data from search results
        "format": "csv" or "json"
    }
    
    Returns:
        JSON response with file path or error
    """
    try:
        data = request.json
        
        if not data or 'data' not in data:
            return jsonify({
                'status': 'error',
                'message': 'No data provided'
            }), 400
        
        export_format = data.get('format', 'csv').lower()
        patent_data = data.get('data', [])
        
        if not patent_data:
            return jsonify({
                'status': 'warning',
                'message': 'No data to export'
            })
        
        # Create dataframe from patent data
        df = pd.DataFrame(patent_data)
        
        # Create exports directory if it doesn't exist
        os.makedirs('exports', exist_ok=True)
        
        # Generate filename
        filename = f"patent_export_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}"
        
        if export_format == 'csv':
            filepath = f"exports/{filename}.csv"
            df.to_csv(filepath, index=False)
        else:
            filepath = f"exports/{filename}.json"
            df.to_json(filepath, orient='records', indent=2)
        
        return jsonify({
            'status': 'success',
            'message': f'Data exported as {export_format.upper()}',
            'filepath': filepath
        })
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': f'Export failed: {str(e)}'
        }), 500

@app.route('/api/analyze', methods=['POST'])
def analyze_patents():
    """
    API endpoint to perform basic analysis on patent data.
    
    Expected JSON body:
    {
        "data": [...]  # Patent data from search results
    }
    
    Returns:
        JSON response with analysis results
    """
    try:
        data = request.json
        
        if not data or 'data' not in data:
            return jsonify({
                'status': 'error',
                'message': 'No data provided'
            }), 400
        
        patent_data = data.get('data', [])
        
        if not patent_data:
            return jsonify({
                'status': 'warning',
                'message': 'No data to analyze'
            })
        
        # Create dataframe from patent data
        df = pd.DataFrame(patent_data)
        
        # Perform basic analysis
        analysis = {}
        
        # Publication trend by year
        if 'Publication date' in df.columns:
            df['Publication year'] = df['Publication date'].str[:4]
            pub_trend = df['Publication year'].value_counts().sort_index().to_dict()
            analysis['publication_trend'] = pub_trend
        
        # Top applicants
        if 'Applicants' in df.columns:
            # Split applicants by semicolon and count occurrences
            all_applicants = []
            for applicants in df['Applicants'].dropna():
                all_applicants.extend([app.strip() for app in applicants.split(';')])
            
            top_applicants = pd.Series(all_applicants).value_counts().head(10).to_dict()
            analysis['top_applicants'] = top_applicants
        
        # Technology distribution (based on IPC)
        if 'IPC' in df.columns:
            # Get main IPC class (first character)
            df['IPC_main'] = df['IPC'].str[:1]
            ipc_dist = df['IPC_main'].value_counts().to_dict()
            analysis['ipc_distribution'] = ipc_dist
        
        return jsonify({
            'status': 'success',
            'analysis': analysis
        })
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': f'Analysis failed: {str(e)}'
        }), 500

if __name__ == '__main__':
    app.run(debug=True, port=5000)

In [85]:
import requests
import pandas as pd
import time
from typing import Dict, List, Optional, Union
import os
from dotenv import load_dotenv
from urllib.parse import quote

# Load environment variables
load_dotenv()

class EPOPatentCollector:
    """
    Complete implementation of EPO Patent Collector with all requested fields
    """
    
    def __init__(self):
        self.base_url = "https://ops.epo.org/3.2"
        self.token_url = f"{self.base_url}/auth/accesstoken"
        self.consumer_key = os.getenv("EPO_CONSUMER_KEY")
        self.consumer_secret = os.getenv("EPO_CONSUMER_SECRET")
        self.access_token = None
        self.token_expiry = 0
        self.headers = {"Accept": "application/json"}
        self.request_delay = 1.2  # 1.2 seconds between requests

    def _get_access_token(self):
        """Manage OAuth 2.0 authentication with retry logic"""
        if time.time() < self.token_expiry and self.access_token:
            return

        try:
            response = requests.post(
                self.token_url,
                data={
                    "grant_type": "client_credentials",
                    "client_id": self.consumer_key,
                    "client_secret": self.consumer_secret
                },
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                timeout=10
            )
            response.raise_for_status()
            token_data = response.json()
            self.access_token = token_data["access_token"]
            self.token_expiry = time.time() + 3300  # 55 minutes
            self.headers["Authorization"] = f"Bearer {self.access_token}"
        except requests.exceptions.RequestException as e:
            print(f"Authentication failed: {str(e)}")
            raise

    def _make_api_request(self, method: str, endpoint: str, **kwargs) -> Optional[Dict]:
        """Central API request handler with error handling"""
        self._get_access_token()
        url = f"{self.base_url}{endpoint}"
        
        try:
            response = requests.request(
                method,
                url,
                headers=self.headers,
                **kwargs
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 401:
                self.token_expiry = 0
                return self._make_api_request(method, endpoint, **kwargs)
            print(f"API request failed: {str(e)}")
            return None
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {str(e)}")
            return None
        finally:
            time.sleep(self.request_delay)

    def build_search_query(self, 
                          keywords: Union[str, List[str]], 
                          date_range: Optional[str] = None,
                          applicant: Optional[str] = None,
                          classification: Optional[str] = None) -> str:
        """Build valid CQL query string"""
        if isinstance(keywords, list):
            keyword_query = " OR ".join([f'"{kw}"' for kw in keywords])
        else:
            keyword_query = f'"{keywords}"'

        query_parts = [
            f"(ti=({keyword_query}) OR ab=({keyword_query}))"
        ]

        if date_range:
            query_parts.append(f"pd={date_range}")
        if applicant:
            query_parts.append(f'pa="{applicant}"')
        if classification:
            query_parts.append(f"ipc={classification}" if classification.startswith(
                ("A","B","C","D","E","F","G","H")) else f"cpc={classification}")

        return " AND ".join(query_parts)

    def search_patents(self, query: str, max_results: int = 100) -> Optional[List[Dict]]:
        """Search patents and return document numbers"""
        endpoint = "/rest-services/published-data/search"
        params = {
            "q": query,
            "Range": f"1-{max_results}"
        }
        
        response = self._make_api_request("GET", endpoint, params=params)
        if not response:
            return None

        try:
            results = response['ops:world-patent-data']['ops:biblio-search']['ops:search-result']['ops:publication-reference']
            return [{
                'doc_number': f"{res['document-id']['country']['$']}{res['document-id']['doc-number']['$']}{res['document-id']['kind']['$']}",
                'title': res.get('title', [{}])[0].get('$', '')
            } for res in (results if isinstance(results, list) else [results])]
        except KeyError as e:
            print(f"Failed to parse search results: {str(e)}")
            return None

    def get_bibliographic_data(self, doc_number: str) -> Optional[Dict]:
        """Get full bibliographic data"""
        endpoint = f"/rest-services/published-data/publication/docdb/{quote(doc_number)}/biblio"
        return self._make_api_request("GET", endpoint)

    def get_abstract(self, doc_number: str) -> Optional[str]:
        """Get patent abstract"""
        endpoint = f"/rest-services/published-data/publication/docdb/{quote(doc_number)}/abstract"
        response = self._make_api_request("GET", endpoint)
        try:
            return response['ops:world-patent-data']['exchange-documents']['exchange-document']['abstract'][0]['$']
        except (KeyError, TypeError):
            return None

    def get_family_data(self, doc_number: str) -> Optional[Dict]:
        """Get patent family data"""
        endpoint = f"/rest-services/family/publication/docdb/{quote(doc_number)}"
        return self._make_api_request("GET", endpoint)

    def get_citations_data(self, doc_number: str) -> Optional[Dict]:
        """Get citation information"""
        endpoint = f"/rest-services/published-data/publication/docdb/{quote(doc_number)}/biblio"
        response = self._make_api_request("GET", endpoint)
        try:
            return response['ops:world-patent-data']['exchange-documents']['exchange-document']['references-cited']
        except KeyError:
            return None

    def extract_patent_info(self, doc_number: str) -> Dict:
        """Extract all required fields from multiple API calls"""
        patent_info = {'publication_number': doc_number}
        
        # Get bibliographic data
        biblio = self.get_bibliographic_data(doc_number)
        if not biblio:
            return patent_info

        try:
            doc = biblio['ops:world-patent-data']['exchange-documents']['exchange-document']
            biblio_data = doc['bibliographic-data']

            # Title
            titles = biblio_data.get('invention-title', [])
            if isinstance(titles,dict):
                titles=[titles]
                
            patent_info['title'] = next(
                (t['$'] for t in titles if isinstance(t, dict) and t.get('@lang') == 'en'),
                titles[0]['$'] if titles else ''
            )

            # Abstract
            patent_info['abstract'] = self.get_abstract(doc_number)

            # Inventors (English)
            inventors = []
            inventors_data = biblio_data.get('parties', {}).get('inventors', {}).get('inventor', [])
            if isinstance(inventors_data, dict) :
                    inventors_data = [inventors_data]
            for inv in inventors_data : 
                if isinstance (inv , dict) : 
                    inventors.append(inv.get('inventor-name', {}).get('name', {}).get('$', ''))
            patent_info['inventors'] = "; ".join(filter(None, inventors))

            # Applicants (English)
            applicants = []
            applicants_data = biblio_data.get('parties', {}).get('applicants', {}).get('applicant', [])
            if isinstance(applicants_data , dict) : 
                applicants_data = [applicants_data]
            for app in applicants_data:
                if isinstance(app, dict) and app.get('@lang') == 'en':
                    applicants.append(app.get('applicant-name', {}).get('name', {}).get('$', ''))
            patent_info['applicants'] = "; ".join(filter(None, applicants))


            # # priority claims
            # priorities = []
            # priority_claims = biblio_data.get('priority-claims' , {}).get('priority-claim',[])
            # if isinstance(priority_claims, dict): 
            #     priority_claims = [priority_claims]
            # for claim in priority_claims : 
            #     if isinstance(claim , dict) : 
            #         priorities.append(claim['priority-active-indicator']['date']['$'])
            # patent_info['earliest_priority'] = min(priorities) if priorities else None

          
            # Classifications
            ipc_classes = []
            ipc_data = biblio_data.get('classifications-ipcr', {}).get('classification-ipcr', [])
            if isinstance(ipc_data, dict): 
                ipc_data = [ipc_data]
            patent_info['ipc'] = "; ".join([
                ipc['text']['$'] 
                for ipc in ipc_data
                if isinstance(ipc, dict) and 'text' in ipc
            ])

            cpc_classes = []
            cpc_data = biblio_data.get('patent-classifications', {}).get('patent-classification', [])
            if isinstance(cpc_data , dict): 
                cpc_data = [cpc_data]
            patent_info['cpc'] = "; ".join([
                cls['text']['$'] 
                for cls in cpc_data
                if isinstance(cls, dict) and cls.get('classification-scheme', {}).get('@scheme') == 'CPC'
            ])

            # Family Data
            family = self.get_family_data(doc_number)

            if family:
                members = family['ops:world-patent-data']['ops:patent-family'].get('ops:family-member', [])
                if isinstance(members , dict): 
                    members= [members]
                members = members if isinstance(members, list) else [members]
                patent_info['family_members'] = [
                    f"{m['publication-reference']['document-id']['country']['$']}"
                    f"{m['publication-reference']['document-id']['doc-number']['$']}"
                    f"{m['publication-reference']['document-id']['kind']['$']}"
                    for m in members
                ]
                patent_info['family_size'] = len(members)
            else:
                patent_info['family_members'] = []
                patent_info['family_size'] = 0

            # Citations
            citations = self.get_citations_data(doc_number)
            if citations:
                patent_info['citations'] = len(citations.get('citation', []))
                patent_info['citations_ipc'] = "; ".join(set(
                    cit.get('classification-ipcr', {}).get('text', {}).get('$', '')
                    for cit in citations.get('citation', [])
                    if cit.get('classification-ipcr')
                ))
            else:
                patent_info['citations'] = 0
                patent_info['citations_ipc'] = ""

        except KeyError as e:
            print(f"Missing key in API response: {str(e)}")
        
        return patent_info

    def collect_patent_data(self, 
                           keywords: Union[str, List[str]], 
                           max_results: int = 100,
                           **kwargs) -> pd.DataFrame:
        """Main data collection method"""
        query = self.build_search_query(keywords, **kwargs)
        print(f"Executing query: {query}")
        
        patents = self.search_patents(query, max_results)
        if not patents:
            return pd.DataFrame()

        results = []
        for i, patent in enumerate(patents[:max_results], 1):
            print(f"Processing patent {i}/{len(patents)}: {patent['doc_number']}")
            patent_info = self.extract_patent_info(patent['doc_number'])
            patent_info['no'] = i
            results.append(patent_info)
        
        return pd.DataFrame(results)

# Example usage
if __name__ == "__main__":
    collector = EPOPatentCollector()
    
    try:
        df = collector.collect_patent_data(
            keywords=["artificial intelligence", "machine learning"],
            max_results=50
        )
        
        if not df.empty:
            print("\nCollected data columns:", df.columns.tolist())
            print("\nSample data:")
            print(df[['no', 'publication_number', 'title', 'applicants', 'citations']].head())
            df.to_csv("patent_data.csv", index=False)
            print("\nData saved to patent_data.csv")
        else:
            print("No data collected")
    except Exception as e:
        print(f"Error: {str(e)}")

Executing query: (ti=("artificial intelligence" OR "machine learning") OR ab=("artificial intelligence" OR "machine learning"))
Processing patent 1/50: KR20250042003A
Error: list indices must be integers or slices, not str


In [86]:
import requests
import pandas as pd
import time
from typing import Dict, List, Optional, Union
import os
from dotenv import load_dotenv
from urllib.parse import quote

# Load environment variables
load_dotenv()

# Global configuration and state
BASE_URL = "https://ops.epo.org/3.2"
TOKEN_URL = f"{BASE_URL}/auth/accesstoken"
CONSUMER_KEY = os.getenv("EPO_CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("EPO_CONSUMER_SECRET")
ACCESS_TOKEN = None
TOKEN_EXPIRY = 0
HEADERS = {"Accept": "application/json"}
REQUEST_DELAY = 1.2  # seconds between requests

def get_access_token() -> None:
    """
    Manage OAuth 2.0 authentication with retry logic.
    Refreshes the token if expired.
    """
    global ACCESS_TOKEN, TOKEN_EXPIRY, HEADERS
    if time.time() < TOKEN_EXPIRY and ACCESS_TOKEN:
        return
    try:
        response = requests.post(
            TOKEN_URL,
            data={
                "grant_type": "client_credentials",
                "client_id": CONSUMER_KEY,
                "client_secret": CONSUMER_SECRET
            },
            headers={"Content-Type": "application/x-www-form-urlencoded"},
            timeout=10
        )
        response.raise_for_status()
        token_data = response.json()
        ACCESS_TOKEN = token_data["access_token"]
        TOKEN_EXPIRY = time.time() + 3300  # 55 minutes
        HEADERS["Authorization"] = f"Bearer {ACCESS_TOKEN}"
    except requests.exceptions.RequestException as e:
        print(f"Authentication failed: {str(e)}")
        raise

def make_api_request(method: str, endpoint: str, **kwargs) -> Optional[Dict]:
    """
    Central API request handler with error handling.
    Automatically refreshes the access token on 401 errors.
    """
    get_access_token()
    url = f"{BASE_URL}{endpoint}"
    try:
        response = requests.request(
            method,
            url,
            headers=HEADERS,
            **kwargs
        )
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 401:
            global TOKEN_EXPIRY
            TOKEN_EXPIRY = 0
            return make_api_request(method, endpoint, **kwargs)
        print(f"API request failed: {str(e)}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {str(e)}")
        return None
    finally:
        time.sleep(REQUEST_DELAY)

def build_search_query(
    keywords: Union[str, List[str]],
    date_range: Optional[str] = None,
    applicant: Optional[str] = None,
    classification: Optional[str] = None
) -> str:
    """
    Build a valid CQL query string for patent search.
    """
    if isinstance(keywords, list):
        keyword_query = " OR ".join([f'"{kw}"' for kw in keywords])
    else:
        keyword_query = f'"{keywords}"'

    query_parts = [f"(ti=({keyword_query}) OR ab=({keyword_query}))"]

    if date_range:
        query_parts.append(f"pd={date_range}")
    if applicant:
        query_parts.append(f'pa="{applicant}"')
    if classification:
        query_parts.append(f"ipc={classification}" if classification.startswith(
            ("A", "B", "C", "D", "E", "F", "G", "H")) else f"cpc={classification}")

    return " AND ".join(query_parts)

def search_patents(query: str, max_results: int = 100) -> Optional[List[Dict]]:
    """
    Search patents using the provided query and return a list of documents.
    """
    endpoint = "/rest-services/published-data/search"
    params = {
        "q": query,
        "Range": f"1-{max_results}"
    }
    response = make_api_request("GET", endpoint, params=params)
    if not response:
        return None

    try:
        results = response['ops:world-patent-data']['ops:biblio-search']['ops:search-result']['ops:publication-reference']
        if isinstance(results, list):
            return [{
                'doc_number': f"{res['document-id']['country']['$']}{res['document-id']['doc-number']['$']}{res['document-id']['kind']['$']}",
                'title': res.get('title', [{}])[0].get('$', '')
            } for res in results]
        else:
            # Handle a single result
            res = results
            return [{
                'doc_number': f"{res['document-id']['country']['$']}{res['document-id']['doc-number']['$']}{res['document-id']['kind']['$']}",
                'title': res.get('title', [{}])[0].get('$', '')
            }]
    except KeyError as e:
        print(f"Failed to parse search results: {str(e)}")
        return None

def get_bibliographic_data(doc_number: str) -> Optional[Dict]:
    """
    Retrieve full bibliographic data for a given patent document.
    """
    endpoint = f"/rest-services/published-data/publication/docdb/{quote(doc_number)}/biblio"
    return make_api_request("GET", endpoint)

def get_abstract(doc_number: str) -> Optional[str]:
    """
    Retrieve the patent abstract.
    """
    endpoint = f"/rest-services/published-data/publication/docdb/{quote(doc_number)}/abstract"
    response = make_api_request("GET", endpoint)
    try:
        doc = response['ops:world-patent-data']['exchange-documents']['exchange-document']
        if isinstance(doc, list):
            doc = doc[0]
        return doc['abstract'][0]['$']
    except (KeyError, TypeError):
        return None

def get_family_data(doc_number: str) -> Optional[Dict]:
    """
    Retrieve patent family data.
    """
    endpoint = f"/rest-services/family/publication/docdb/{quote(doc_number)}"
    return make_api_request("GET", endpoint)

def get_citations_data(doc_number: str) -> Optional[Dict]:
    """
    Retrieve patent citations information.
    """
    endpoint = f"/rest-services/published-data/publication/docdb/{quote(doc_number)}/biblio"
    response = make_api_request("GET", endpoint)
    try:
        doc = response['ops:world-patent-data']['exchange-documents']['exchange-document']
        if isinstance(doc, list):
            doc = doc[0]
        return doc['references-cited']
    except KeyError:
        return None

def extract_patent_info(doc_number: str) -> Dict:
    """
    Extract detailed patent information by combining multiple API calls.
    """
    patent_info = {'publication_number': doc_number}
    biblio = get_bibliographic_data(doc_number)
    if not biblio:
        return patent_info

    try:
        doc = biblio['ops:world-patent-data']['exchange-documents']['exchange-document']
        if isinstance(doc, list):
            doc = doc[0]
        biblio_data = doc['bibliographic-data']

        # Title
        titles = biblio_data.get('invention-title', [])
        if isinstance(titles, dict):
            titles = [titles]
        patent_info['title'] = next(
            (t['$'] for t in titles if isinstance(t, dict) and t.get('@lang') == 'en'),
            titles[0]['$'] if titles else ''
        )

        # Abstract
        patent_info['abstract'] = get_abstract(doc_number)

        # Inventors (English)
        inventors = []
        inventors_data = biblio_data.get('parties', {}).get('inventors', {}).get('inventor', [])
        if isinstance(inventors_data, dict):
            inventors_data = [inventors_data]
        for inv in inventors_data:
            if isinstance(inv, dict):
                inventors.append(inv.get('inventor-name', {}).get('name', {}).get('$', ''))
        patent_info['inventors'] = "; ".join(filter(None, inventors))

        # Applicants (English)
        applicants = []
        applicants_data = biblio_data.get('parties', {}).get('applicants', {}).get('applicant', [])
        if isinstance(applicants_data, dict):
            applicants_data = [applicants_data]
        for app in applicants_data:
            if isinstance(app, dict) and app.get('@lang') == 'en':
                applicants.append(app.get('applicant-name', {}).get('name', {}).get('$', ''))
        patent_info['applicants'] = "; ".join(filter(None, applicants))

        # Classifications
        ipc_data = biblio_data.get('classifications-ipcr', {}).get('classification-ipcr', [])
        if isinstance(ipc_data, dict):
            ipc_data = [ipc_data]
        patent_info['ipc'] = "; ".join([
            ipc['text']['$']
            for ipc in ipc_data
            if isinstance(ipc, dict) and 'text' in ipc
        ])

        cpc_data = biblio_data.get('patent-classifications', {}).get('patent-classification', [])
        if isinstance(cpc_data, dict):
            cpc_data = [cpc_data]
        patent_info['cpc'] = "; ".join([
            cls['text']['$']
            for cls in cpc_data
            if isinstance(cls, dict) and cls.get('classification-scheme', {}).get('@scheme') == 'CPC'
        ])

        # Family Data
        family = get_family_data(doc_number)
        if family:
            members = family['ops:world-patent-data']['ops:patent-family'].get('ops:family-member', [])
            if isinstance(members, dict):
                members = [members]
            patent_info['family_members'] = [
                f"{m['publication-reference']['document-id']['country']['$']}"
                f"{m['publication-reference']['document-id']['doc-number']['$']}"
                f"{m['publication-reference']['document-id']['kind']['$']}"
                for m in members
            ]
            patent_info['family_size'] = len(members)
        else:
            patent_info['family_members'] = []
            patent_info['family_size'] = 0

        # Citations
        citations = get_citations_data(doc_number)
        if citations:
            citations_list = citations.get('citation', [])
            if isinstance(citations_list, dict):
                citations_list = [citations_list]
            patent_info['citations'] = len(citations_list)
            patent_info['citations_ipc'] = "; ".join(set(
                cit.get('classification-ipcr', {}).get('text', {}).get('$', '')
                for cit in citations_list
                if cit.get('classification-ipcr')
            ))
        else:
            patent_info['citations'] = 0
            patent_info['citations_ipc'] = ""

    except KeyError as e:
        print(f"Missing key in API response: {str(e)}")
    
    return patent_info

def collect_patent_data(keywords: Union[str, List[str]], max_results: int = 100, **kwargs) -> pd.DataFrame:
    """
    Main data collection function that builds the query, retrieves patent data,
    and returns the results as a Pandas DataFrame.
    """
    query = build_search_query(keywords, **kwargs)
    print(f"Executing query: {query}")
    
    patents = search_patents(query, max_results)
    if not patents:
        return pd.DataFrame()

    results = []
    for i, patent in enumerate(patents[:max_results], 1):
        print(f"Processing patent {i}/{len(patents)}: {patent['doc_number']}")
        patent_info = extract_patent_info(patent['doc_number'])
        patent_info['no'] = i
        results.append(patent_info)
    
    return pd.DataFrame(results)

# Example usage:
if __name__ == "__main__":
    try:
        df = collect_patent_data(
            keywords=["artificial intelligence", "machine learning"],
            max_results=50
        )
        if not df.empty:
            print("\nCollected data columns:", df.columns.tolist())
            print("\nSample data:")
            print(df[['no', 'publication_number', 'title', 'applicants', 'citations']].head())
            df.to_csv("patent_data.csv", index=False)
            print("\nData saved to patent_data.csv")
        else:
            print("No data collected")
    except Exception as e:
        print(f"Error: {str(e)}")


Executing query: (ti=("artificial intelligence" OR "machine learning") OR ab=("artificial intelligence" OR "machine learning"))
Processing patent 1/50: KR20250042003A
Error: list indices must be integers or slices, not str


In [ ]:
import os
import requests
import time
from urllib.parse import quote
import pandas as pd
from dotenv import load_dotenv
import xml.etree.ElementTree as ET

# Constants for API endpoints
TOKEN_URL = "https://ops.epo.org/3.2/auth/accesstoken"
BASE_URL = "https://ops.epo.org/3.2/rest-services"

# Global token cache variables
TOKEN = None
TOKEN_EXPIRY = 0

# Load credentials from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY").strip()
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET").strip()

def get_access_token() -> str:
    """
    Get or refresh the OAuth access token.
    """
    global TOKEN, TOKEN_EXPIRY
    if TOKEN and time.time() < TOKEN_EXPIRY:
        return TOKEN
    data = {
        "grant_type": "client_credentials",
        "client_id": CONSUMER_KEY,
        "client_secret": CONSUMER_SECRET
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    response = requests.post(TOKEN_URL, data=data, headers=headers, timeout=15)
    response.raise_for_status()
    TOKEN = response.json()["access_token"]
    TOKEN_EXPIRY = time.time() + 3500  # about 58 minutes validity
    return TOKEN

def parse_english_abstract(xml_content: str) -> str:
    """
    Parse XML content and extract the English abstract.
    
    Expected structure:
      <abstract lang="en">
         <p>Text of abstract...</p>
      </abstract>
    """
    try:
        root = ET.fromstring(xml_content)
        # Define the default namespace as used in the XML
        ns = {'default': 'http://www.epo.org/exchange'}
        # Look for <abstract> elements in the default namespace
        abstracts = root.findall('.//default:abstract', ns)
        for abstract in abstracts:
            # Check the language attribute from the XML namespace for lang (xml:lang)
            if abstract.attrib.get('{http://www.w3.org/XML/1998/namespace}lang') == 'en':
                # Join text from all child <p> elements
                return ' '.join(p.text.strip() for p in abstract.findall('default:p', ns) if p.text)
        return None
    except ET.ParseError as e:
        print(f"XML parsing error: {e}")
        return None

def get_english_abstract(publication_number: str) -> str:
    """
    Retrieve the English abstract for a given publication number using the OPS abstract endpoint.
    
    For example, for publication_number = 'KR102511398B1',
    the endpoint URL becomes:
      https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/KR102511398B1/abstract
    """
    try:
        token = get_access_token()
        url = f"{BASE_URL}/published-data/publication/docdb/{quote(publication_number)}/abstract"
        headers = {
            "Authorization": f"Bearer {token}",
            "Accept": "application/xml"
        }
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code in (403, 404):
            print(f"Patent {publication_number} - Access forbidden or not found ({response.status_code})")
            return None

        response.raise_for_status()
        # Uncomment the next line to print raw XML during debugging
        # print(response.text[:300])
        return parse_english_abstract(response.text)
    except Exception as e:
        print(f"Failed to retrieve abstract for {publication_number}: {e}")
        return None

def add_abstracts_to_dataframe(df: pd.DataFrame, patent_col: str) -> pd.DataFrame:
    """
    Enrich the DataFrame by adding an 'english_abstract' column.
    Each row will query the OPS API for its abstract based on the publication number.
    """
    if patent_col not in df.columns:
        raise ValueError(f"Column '{patent_col}' not found in DataFrame")
    
    result_df = df.copy()
    abstracts = []
    total = len(result_df)
    for idx, pub_number in enumerate(result_df[patent_col]):
        print(f"Processing {idx+1}/{total}: {pub_number}")
        abstract = get_english_abstract(pub_number)
        abstracts.append(abstract)
        # Delay to respect OPS rate limits
        time.sleep(1.2)
    result_df['english_abstract'] = abstracts
    return result_df

# ======= USAGE EXAMPLE =======
if __name__ == "__main__":
    # Sample DataFrame; replace with your actual DataFrame
    # df = pd.DataFrame({
    #     'first publication number': [
    #         'KR102511398B1',   # Example from your XML
    #         'EP2840123A1', 
    #         'US2006250902A1'
    #     ],
    #     'title': ['Title A', 'Title B', 'Title C']
    # })
    
    # Process the DataFrame to add English abstracts
    enriched_df = add_abstracts_to_dataframe(df, 'first publication number')
    
    # Display a preview of the results
    enriched_df[['first publication number', 'english_abstract']]
    
    # Optionally, save the results to CSV for use by your frontend
    enriched_df.to_csv("patents_with_abstracts.csv", index=False)


Executing query: (ti=("artificial intelligence" OR "machine learning") OR ab=("artificial intelligence" OR "machine learning"))
Authentication failed: 401 Client Error: Unauthorized for url: https://ops.epo.org/3.2/auth/accesstoken
Error: 401 Client Error: Unauthorized for url: https://ops.epo.org/3.2/auth/accesstoken


In [31]:
df=df.head(100)

In [32]:
import os
import requests
import time
from urllib.parse import quote
import pandas as pd
from dotenv import load_dotenv
import xml.etree.ElementTree as ET

# Constants for API endpoints
TOKEN_URL = "https://ops.epo.org/3.2/auth/accesstoken"
BASE_URL = "https://ops.epo.org/3.2/rest-services"

# Global token cache variables
TOKEN = None
TOKEN_EXPIRY = 0

# Load credentials from .env file
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY").strip()
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET").strip()

def get_access_token() -> str:
    """
    Get or refresh the OAuth access token.
    """
    global TOKEN, TOKEN_EXPIRY
    if TOKEN and time.time() < TOKEN_EXPIRY:
        return TOKEN
    data = {
        "grant_type": "client_credentials",
        "client_id": CONSUMER_KEY,
        "client_secret": CONSUMER_SECRET
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    response = requests.post(TOKEN_URL, data=data, headers=headers, timeout=15)
    response.raise_for_status()
    TOKEN = response.json()["access_token"]
    TOKEN_EXPIRY = time.time() + 3500  # about 58 minutes validity
    return TOKEN

def parse_english_abstract(xml_content: str) -> str:
    """
    Parse XML content and extract the English abstract.
    
    Expected structure:
      <abstract lang="en">
         <p>Text of abstract...</p>
      </abstract>
    """
    try:
        root = ET.fromstring(xml_content)
        # Define the default namespace as used in the XML
        ns = {'default': 'http://www.epo.org/exchange'}
        # Look for <abstract> elements in the default namespace
        abstracts = root.findall('.//default:abstract', ns)
        for abstract in abstracts:
            # Check the language attribute from the XML namespace for lang (xml:lang)
            if abstract.attrib.get('{http://www.w3.org/XML/1998/namespace}lang') == 'en':
                # Join text from all child <p> elements
                return ' '.join(p.text.strip() for p in abstract.findall('default:p', ns) if p.text)
        return None
    except ET.ParseError as e:
        print(f"XML parsing error: {e}")
        return None

def get_english_abstract(publication_number: str) -> str:
    """
    Retrieve the English abstract for a given publication number using the OPS abstract endpoint.
    
    For example, for publication_number = 'KR102511398B1',
    the endpoint URL becomes:
      https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/KR102511398B1/abstract
    """
    try:
        token = get_access_token()
        url = f"{BASE_URL}/published-data/publication/docdb/{quote(publication_number)}/abstract"
        headers = {
            "Authorization": f"Bearer {token}",
            "Accept": "application/xml"
        }
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code in (403, 404):
            print(f"Patent {publication_number} - Access forbidden or not found ({response.status_code})")
            return None

        response.raise_for_status()
        # Uncomment the next line to print raw XML during debugging
        # print(response.text[:300])
        return parse_english_abstract(response.text)
    except Exception as e:
        print(f"Failed to retrieve abstract for {publication_number}: {e}")
        return None

def add_abstracts_to_dataframe(df: pd.DataFrame, patent_col: str) -> pd.DataFrame:
    """
    Enrich the DataFrame by adding an 'english_abstract' column.
    Each row will query the OPS API for its abstract based on the publication number.
    """
    if patent_col not in df.columns:
        raise ValueError(f"Column '{patent_col}' not found in DataFrame")
    
    result_df = df.copy()
    abstracts = []
    total = len(result_df)
    for idx, pub_number in enumerate(result_df[patent_col]):
        print(f"Processing {idx+1}/{total}: {pub_number}")
        abstract = get_english_abstract(pub_number)
        abstracts.append(abstract)
        # Delay to respect OPS rate limits
        time.sleep(1.2)
    result_df['english_abstract'] = abstracts
    return result_df

# ======= USAGE EXAMPLE =======
if __name__ == "__main__":
    # Sample DataFrame; replace with your actual DataFrame
    # df = pd.DataFrame({
    #     'first publication number': [
    #         'KR102511398B1',   # Example from your XML
    #         'EP2840123A1', 
    #         'US2006250902A1'
    #     ],
    #     'title': ['Title A', 'Title B', 'Title C']
    # })
    
    # Process the DataFrame to add English abstracts
    enriched_df = add_abstracts_to_dataframe(df, 'first publication number')
    
    # Display a preview of the results
    enriched_df[['first publication number', 'english_abstract']]
    
    # Optionally, save the results to CSV for use by your frontend
    enriched_df.to_csv("patents_with_abstracts.csv", index=False)


Processing 1/100: US2006250902A1
Processing 2/100: KR102511398B1
Processing 3/100: KR102511391B1
Processing 4/100: KR20230163874A
Processing 5/100: GB2631101A
Processing 6/100: CN105098267A
Processing 7/100: US2022140365A1
Processing 8/100: WO2011019133A2
Processing 9/100: TW201310755A
Processing 10/100: CN108550935A
Processing 11/100: CN112606713A
Processing 12/100: US2013093378A1
Processing 13/100: CN109263516A
Processing 14/100: CN110015205A
Processing 15/100: CN214240460U
Processing 16/100: CN206968376U
Processing 17/100: CN106025314A
Processing 18/100: WO2023128748A1
Processing 19/100: CN204109822U
Processing 20/100: JPH11162523A
Processing 21/100: CN115431743A
Processing 22/100: KR20210053589A
Processing 23/100: CN113525181A
Processing 24/100: CN113085622A
Processing 25/100: US12005772B2
Processing 26/100: CN101685890A
Processing 27/100: CN108674229A
Processing 28/100: CN117124905A
Processing 29/100: CN110014885A
Processing 30/100: US2023256848A1
Processing 31/100: CN115759606A
P

In [34]:
enriched_df['english_abstract']

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Name: english_abstract, Length: 100, dtype: object

In [40]:
import os
import requests
from urllib.parse import quote
from dotenv import load_dotenv
import json

# Load credentials
load_dotenv()
CONSUMER_KEY = os.getenv("CONSUMER_KEY").strip()
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET").strip()

TOKEN_URL = "https://ops.epo.org/3.2/auth/accesstoken"
BASE_URL = "https://ops.epo.org/3.2/rest-services"

def get_access_token():
    data = {
        "grant_type": "client_credentials",
        "client_id": CONSUMER_KEY,
        "client_secret": CONSUMER_SECRET
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    response = requests.post(TOKEN_URL, data=data, headers=headers, timeout=15)
    response.raise_for_status()
    return response.json()["access_token"]

def get_abstract_json(publication_number: str) -> dict:
    """
    Retrieve the abstract for a given publication number as JSON.
    
    For example, if publication_number = 'KR102511398B1', the endpoint URL becomes:
      https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/KR102511398B1/abstract
    """
    token = get_access_token()
    url = f"{BASE_URL}/published-data/publication/docdb/{quote(publication_number)}/abstract"
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }
    response = requests.get(url, headers=headers, timeout=15)
    print(f"Status code: {response.status_code}")
    response.raise_for_status()
    return response.json()

if __name__ == "__main__":
    # Replace with a valid publication number
    publication_number = "KR102511398B1"
    try:
        abstract_json = get_abstract_json(publication_number)
        print("Abstract JSON result:\n")
        print(json.dumps(abstract_json, indent=2))
    except Exception as e:
        print(f"Error: {e}")


Status code: 200
Abstract JSON result:

{
  "ops:world-patent-data": {
    "@xmlns": {
      "ops": "http://ops.epo.org",
      "$": "http://www.epo.org/exchange",
      "xlink": "http://www.w3.org/1999/xlink"
    },
    "exchange-documents": {
      "exchange-document": {
        "@country": "KR",
        "@doc-number": "102511398",
        "@kind": "B1",
        "bibliographic-data": {
          "publication-reference": {
            "document-id": [
              {
                "@document-id-type": "docdb",
                "country": {
                  "$": "KR"
                },
                "doc-number": {
                  "$": "102511398"
                },
                "kind": {
                  "$": "B1"
                },
                "date": {
                  "$": "20230320"
                }
              },
              {
                "@document-id-type": "epodoc",
                "doc-number": {
                  "$": "KR102511398B"
                },

In [45]:
def extract_english_abstract_from_json(json_data: dict) -> str:
    """
    Given the JSON response from the OPS abstract endpoint, extract and return the English abstract text.
    
    Expected JSON structure (based on your example):
    {
      "ops:world-patent-data": {
        ...
        "exchange-documents": {
          "exchange-document": {
            ...
            "abstract": [
              {
                "@lang": "en",
                "p": {"$": "The English abstract text..."}
              },
              {
                "@lang": "ol",
                "p": {"$": "Other language text..."}
              }
            ]
          }
        }
      }
    }
    
    Returns the English abstract as a string, or None if not found.
    """
    try:
        world_data = json_data.get("ops:world-patent-data", {})
        exch_docs = world_data.get("exchange-documents", {})
        doc = exch_docs.get("exchange-document", {})
        abstract_section = doc.get("abstract", [])
        # If abstract is a dict (only one present), convert it to a list
        if isinstance(abstract_section, dict):
            abstract_section = [abstract_section]
        for abstract in abstract_section:
            if abstract.get("@lang") == "en":
                # The abstract text is in the "p" field as a dict with key "$"
                p = abstract.get("p")
                if isinstance(p, dict):
                    return p.get("$", "").strip()
                elif isinstance(p, list):
                    # In case of multiple paragraphs
                    return " ".join(item.get("$", "").strip() for item in p if isinstance(item, dict))
                elif isinstance(p, str):
                    return p.strip()
        return None
    except Exception as e:
        print("Error extracting English abstract:", e)
        return None


In [ ]:
def add_abstracts_to_dataframe(df: pd.DataFrame, patent_col: str) -> pd.DataFrame:
    """
    Enrich the DataFrame by adding an 'english_abstract' column.
    For each patent number in the patent_col, this function:
      1. Retrieves the abstract JSON from the OPS API.
      2. Extracts the English abstract using extract_english_abstract_from_json().
      3. Stores the result in a new column.
      
    A delay is added between requests to comply with rate limits.
    """
    if patent_col not in df.columns:
        raise ValueError(f"Column '{patent_col}' not found in DataFrame")
    
    abstracts = []
    total = len(df)
    for idx, pub_number in enumerate(df[patent_col]):
        print(f"Processing {idx+1}/{total}: {pub_number}")
        try:
            json_data = get_abstract_json(pub_number)
            abstract_text = extract_english_abstract_from_json(json_data)
        except Exception as e:
            print(f"Error processing {pub_number}: {e}")
            abstract_text = None
        abstracts.append(abstract_text)
        # Pause to respect rate limits (adjust delay as needed)
        time.sleep(1.2)
    df = df.copy()
    df["english_abstract"] = abstracts
    return df

# === USAGE EXAMPLE ===
if __name__ == "__main__":
    # Sample DataFrame with a column "first publication number"
    # Replace this sample with your actual DataFrame.
    # sample_data = {
    #     "first publication number": [
    #         "KR102511398B1", 
    #         "EP2840123A1", 
    #         "US2006250902A1"
    #     ],
    #     "title": ["Title A", "Title B", "Title C"]
    # }
    df = pd.DataFrame(sample_data)
    
    # Process the DataFrame to add the "english_abstract" column
    enriched_df = add_abstracts_to_dataframe(df, "first publication number")
    
    # Display results and optionally save to CSV
    print(enriched_df[["first publication number", "english_abstract"]])
    enriched_df.to_csv("patents_with_abstracts.csv", index=False)

Processing 1/3: KR102511398B1
Processing 2/3: EP2840123A1
Error processing EP2840123A1: 404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/docdb/EP2840123A1/abstract
Processing 3/3: US2006250902A1
  first publication number                                   english_abstract
0            KR102511398B1  The present invention relates to a vehicle cha...
1              EP2840123A1                                               None
2           US2006250902A1  This invention relates to plug-in hybrid propu...


In [47]:
import json


# Convert string to dictionary
json_data = get_abstract_json("KR102511398B1")

# Extract the English abstract
english_abstract = extract_english_abstract_from_json(json_data)
print("Extracted English Abstract:")
print(english_abstract)


Extracted English Abstract:
The present invention relates to a vehicle charging system for hydrogen and electricity charging, which can promote the activation of an electric vehicle and a hydrogen vehicle. The vehicle charging system comprises: an electric vehicle charger; a hydrogen vehicle charger; a hydrogen fuel cell; a hydrogen tank; a battery; a pressurizer; a controller; a case; a support frame; a mounting frame; a hydrogen distribution module; a fixing guide groove; a fixing guide bar; a fixing bar; a fixing groove; and a fixing frame.


In [44]:
def print_json_structure(json_data: dict, indent: int = 2):
    import json
    print(json.dumps(json_data, indent=indent))
    
if __name__ == "__main__":
    publication_number = "KR102511398B1"
    try:
        abstract_json = get_abstract_json(publication_number)  # from our previous code
        print_json_structure(abstract_json)
    except Exception as e:
        print(f"Error: {e}")


{
  "ops:world-patent-data": {
    "@xmlns": {
      "ops": "http://ops.epo.org",
      "$": "http://www.epo.org/exchange",
      "xlink": "http://www.w3.org/1999/xlink"
    },
    "exchange-documents": {
      "exchange-document": {
        "@country": "KR",
        "@doc-number": "102511398",
        "@kind": "B1",
        "bibliographic-data": {
          "publication-reference": {
            "document-id": [
              {
                "@document-id-type": "docdb",
                "country": {
                  "$": "KR"
                },
                "doc-number": {
                  "$": "102511398"
                },
                "kind": {
                  "$": "B1"
                },
                "date": {
                  "$": "20230320"
                }
              },
              {
                "@document-id-type": "epodoc",
                "doc-number": {
                  "$": "KR102511398B"
                },
                "date": {
             